In [ ]:
import re
import json
import time
import random
import argparse
import requests
from dataclasses import dataclass
from typing import Optional, List

In [4]:
@dataclass
class IdeaLabLLMOpenAI:
    # api_url: str = "https://idealab.xxx"
    api_url: str = "https://aliyuque.xxx",
    api_key: str = "xxx"
    model_name: str = "gpt-4o-0513"
    max_tokens: int = 4096
    temperature: float = 0

    @property
    def _llm_type(self) -> str:
        return f"IdeaLabLLM {self.model_name}"

    def _call(self,
              prompt: str,
            #   stop: Optional[List[str]] = None,
            #   run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs):
        data = {
            "model": self.model_name,
            "prompt": prompt,
            "maxTokens": self.max_tokens,
            "temperature": self.temperature
        }

        headers = {"X-AK": f"{self.api_key}", "accept": "*/*", "Content-Type": "application/json"}

        data = {
            "platformInput": {
                "model": self.model_name
            }, "messages": [
                {
                    "role": "user",
                    "content": prompt
                }], "toolChoice": "auto"
        }

        response = requests.post(self.api_url, json=data, headers=headers, timeout=1500).json()

        if response['success']:
            # print(response)
            response_data = response['data']['choices'][0]["message"]["content"]
            return response_data
        else:
            raise Exception(f"Failed to call API: {response['success']} {response}")

In [5]:
# If not intern at Alibaba, you may need to change this part, e.g., use openai to prompt GPT-4o.
llm = IdeaLabLLMOpenAI(api_url = "https://idealab.alibabaxxx",
                        api_key = "xxx",
                        model_name = 'gpt-4o-0513',
                        max_tokens = 4096,
                        temperature = 0)

## **P-code Plan Generation**

In [ ]:
extract_plan_prompt = "Given the [Task Description], [Task] and [Solution Trajectory], you should extract the step-by-step [Plan] for solving the task. Please note that the extracted [Plan] should be global and do not contain the information from \"Observation\" part of [Solution Trajectory]. Your extracted [Plan] must strictly follow the pseudo code format and output in this format:\nStep 1. ... \nStep 2. ... \nStep 3. ... \n....\n"

### **ALFWorld**

In [4]:
def getTaskType(task):
    if "Your task is to: clean" in task:
        return 'clean'
    elif "Your task is to: cool" in task: 
        return 'cool'
    elif "Your task is to: heat" in task:
        return 'heat'
    elif "Your task is to: examine" in task or "Your task is to: look at" in task:
        return 'examine'
    elif "Your task is to: put" in task or "Your task is to: find two" in task: # put one & put two
        return 'put'
    else:
        print('loss')

In [ ]:
icl_example = "Here is one example.\n<sys_prompt>\n\n[Task]:\n<task>\n\n[Solution Trajectory]:\n<traj>\n\n[Plan]:\n<plan>\n\nNow is your turn.\n" 
example_task = {"put": "You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a shelf 2, a shelf 1, and a sidetable 1.\nYour task is to: put some alarmclock on desk.", \
"clean": "You are in the middle of a room. Looking quickly around you, you see a cabinet 21, a cabinet 20, a cabinet 19, a cabinet 18, a cabinet 17, a cabinet 16, a cabinet 15, a cabinet 14, a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: clean some apple and put it in diningtable.", \
"cool": "You are in the middle of a room. Looking quickly around you, you see a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: cool some apple and put it in diningtable.", \
"heat": "You are in the middle of a room. Looking quickly around you, you see a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 1, a diningtable 1, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: heat some apple and put it in diningtable.", \
"examine": "You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 17, a drawer 16, a drawer 15, a drawer 14, a drawer 13, a drawer 12, a drawer 11, a drawer 10, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.\n\nYour task is to: look at alarmclock under the desklamp."
}
example_traj = {"put": "Thought: I need to find an alarmclock and put it on the desk. Alarmclocks usually located on bedside tables or in the drawers. Therefore, I will check the drawers first.\nAction: go to drawer 1\nObservation: The drawer 1 is closed.\nThought: Since the drawer 1 is closed, I need open it.\nAction: open drawer 1\nObservation: You open the drawer 1. The drawer 1 is open. In it, you see a remotecontrol 1.\nThought: Drawer 1 only has a remotecontrol. I need to continue searching the other drawers for the alarmclock.\nAction: go to drawer 2\nObservation: The drawer 2 is closed.\nThought: I need to open the drawer 2.\nAction: open drawer 2\nObservation: You open the drawer 2. The drawer 2 is open. In it, you see a book 1.\n No alarmclock is found in drawer 2. I need continue searching the other drawers for the alarm clock.\nAction: go to drawer 3\nObservation: The drawer 3 is closed.\nThought: I need to open the drawer 3.\nAction: open drawer 3\nObservation: You open the drawer 3. The drawer 3 is open. In it, you see a pencil 1.\nThought: I can't find any alarmclock in the drawers. I'll check the shelves next.\nAction: go to shelf 1\nObservation: On the shelf 1, you see nothing.\nThought: There is no alarmclock in shelf 1. I need to continue searching the other shelves for the alarmclock.\nAction: go to shelf 2\nObservation: On the shelf 2, you see nothing.\nThought: There's no alarmclock on both shelves. Alarmclock might be on the sidetable.\nAction: go to sidetable 1\nObservation: On the sidetable 1, you see a alarmclock 2, a alarmclock 1, a creditcard 1, a desklamp 1, a keychain 1, and a pen 1.\nThought: I have found an alarmclock on sidetable 1. I should take it so I can put it on desk.\nAction: take alarmclock 1 from sidetable 1\nObservation: You pick up the alarmclock 1 from the sidetable 1.\nThought: With alarmclock 1 in hand, I need to go to the desk to place it on.\nAction: go to desk 1\nObservation: On the desk 1, you see a book 2, a bowl 1, a cd 3, a cd 2, a cd 1, a cellphone 1, a keychain 2, and a pencil 2.\nThought: Now I need to place the alarmclock 1 on the desk 1 to complete the task.\nAction: put alarmclock 1 in/on desk 1.", \
"clean": "Thought: I need to find an apple first. It is likely to be stored in one of the cabinets or on the countertop. I will start by checking the countertop 1.\nAction: go to countertop 1\nObservation: On the countertop 1, you see a apple 2, a bowl 3, a bread 2, a butterknife 1, a cup 1, a fork 1, a knife 3, a knife 1, a saltshaker 1, a spatula 2, a spatula 1, a spoon 1, and a tomato 1.\nThought: I see a apple 2 on the countertop 1.  should pick up it to clean.\nAction: take apple 2 from countertop 1\nObservation: You pick up the apple 2 from the countertop 1.\nThought: Now I have the apple, I need to clean it. It is likely to be stored in one of the cabinets or on the countertop. I will start by checking the sinkbasin 1.\nAction: go to sinkbasin 1\nObservation: On the sinkbasin 1, you see a potato 1.\nThought: I'm at the sinkbasin. I'll use the sink to wash the apple.\nAction: clean apple 2 with sinkbasin 1\nObservation: You clean the apple 2 using the sinkbasin 1.\nThought: Now the apple is clean, I need to put it on the diningtable 1.\nAction: go to diningtable 1\nObservation: On the diningtable 1, you see a bowl 1, a bread 1, a cup 3, a dishsponge 1, a knife 2, a lettuce 1, a peppershaker 2, a peppershaker 1, and a spatula 3.\nThought: I've arrived at the diningtable, and I'll place the clean apple here as required.\nAction: put apple 2 in/on diningtable 1.", \
"cool": "Thought: I need to find an apple first. It is likely to be stored in one of the cabinets or on the countertop. I will start by checking the countertop 1.\nAction: go to countertop 1\nObservation: On the countertop 1, you see a apple 2, a butterknife 1, a mug 1, a pan 1, a plate 2, a plate 1, a potato 2, a potato 1, a spatula 1, and a winebottle 2.\nThought: I have found the apple on countertop 1. I should pick it up to proceed with the task.\nAction: take apple 2 from countertop 1\nObservation: You pick up the apple 2 from the countertop 1.\nThought: Now I have the apple, I need to cool it down. The fridge is likely to be the best place to do this.\nAction: go to fridge 1\nObservation: The fridge 1 is closed.\nThought: The fridge is closed. I need to open it to place the apple inside for cooling.\nAction: open fridge 1\nObservation: You open the fridge 1. The fridge 1 is open. In it, you see an bread 1, an egg 1, a lettuce 1, and a pan 2.\nThought: It's time to place the apple in the fridge 1 to cool it.\nAction: cool apple 2 with fridge 1\nObservation: You cool the apple 2 using the fridge 1.\nThought: Now the apple is cool, I need to put it on the dining table.\nAction: go to diningtable 1\nObservation: On the diningtable 1, you see a apple 3, a bowl 2, a bread 1, a butterknife 3, a butterknife 2, a fork 3, a lettuce 2, a lettuce 1, a peppershaker 2, a potato 3, a saltshaker 2, a soapbottle 3, a soapbottle 2, a spatula 2, and a winebottle 1.\nThought: I am at the dining table. I should put the apple down here to complete the task.\nAction: put apple 2 in/on diningtable 1.", \
"heat": "Thought: I need to heat an apple and put it on the dining table. I'll first look for an apple. The best places to find one would be a cabinet or the fridge.\nAction: go to cabinet 1\nObservation: The cabinet 1 is closed.\nThought: I need to open the cabinet 1 to search for apples.\nAction: open cabinet 1\nObservation: You open the cabinet 1. The cabinet 1 is open. In it, you see nothing.\nThought: The apple isn't in cabinet 1. Now I'll check the other common location: the fridge.\nAction: go to fridge 1\nObservation: The fridge 1 is closed.\nThought: I need to open the fridge 1 to search for apples.\nAction: open fridge 1\nObservation: You open the fridge 1. The fridge 1 is open. In it, you see a apple 3, a apple 2, a bread 3, a cup 2, and a tomato 1.\nThought: There are several items in the fridge 1, including apples. I need to pick up an apple to proceed with the task.\nAction: take apple 2 from fridge 1\nObservation: You pick up the apple 2 from the fridge 1.\nThought: Now that I have the apple, I need to find a way to heat it. The microwave is a suitable appliance for heating food quickly.\nAction: go to microwave 1\nObservation: The microwave 1 is closed.\nThought:Before I can heat the apple, I need to open the microwave since it's closed.\nAction: open microwave 1\ nObservation: You open the microwave 1. The microwave 1 is open.\nThought: With the microwave open, I can now place the apple inside and heat it.\nAction: heat apple 2 with microwave 1\nObservation: You heat the apple 2 using the microwave 1.\nThought: The apple is heated, and the next step in the task is to put it on the diningtable. I need to locate one diningtable.\nAction: go to diningtable 1\nObservation: On the diningtable 1, you see a apple 1, a bread 1, a butterknife 1, a dishsponge 1, a fork 2, a fork 1, a knife 1, a lettuce 1, a pan 1, a pen 3, a pen 2, a pen 1, a pencil 1, a plate 1, a potato 1, a spatula 1, a spoon 1, and a winebottle 1.\nThought: I've arrived at the diningtable 1. I should now complete the task by putting the heated apple on the diningtable 1.\nAction: put apple 2 in/on diningtable 1.", \
"examine":"Thought: To complete the task, I need to find the alarm clock first. I think the alarm clock might be on the desk or in the drawers. I should start with the desk.\nAction: go to desk 1 \nObservation: On the desk 1, you see a alarmclock 1, a bowl 1, a cd 2, a cd 1, a mug 2, a mug 1, and a pencil 1.\nThought: I have found an alarm clock on desk 1, so I should pick up the alarm clock.\nAction: take alarmclock 1 from desk 1 \nObservation: You pick up the alarmclock 1 from the desk 1 \nThought: Now I have the alarm clock. I need to find a lamp to view it under. I think a desklamp would be perfect for this task. I should go to the dresser next.\nAction: go to dresser 1 \nObservation: On the dresser 1, you see a alarmclock 2, a desklamp 1, a keychain 3, a keychain 1, and a watch 1.\nThought: I see the desklamp on dresser 1. I will use it to look at the alarmclock. Action: use desklamp 1"
}
example_plan = {"put": "Step 1. alarmclock_name, location_name = locate_an_alarmclock(candidate_locations);\nStep 2. take_an_alarmclock(alarmclock_name, location_name);\nStep 3. go_to_the_desk();\nStep 4. put_alarmclock_on_desk().", \
"clean": "Step 1. apple_name, location_name = locate_an_apple(candidate_locations);\nStep 2. take_an_apple(apple_name, location_name);\nStep 3. go_to_the_sink();\nStep 4. clean_apple_using_sink();\nStep 5. go_to_the_diningtable();\nStep 6. put_cleaned_apple_on_diningtable().", \
"cool": "Step 1. apple_name, location_name = locate_an_apple(candidate_locations);\nStep 2. take_an_apple(apple_name, location_name);\nStep 3. go_to_the_fridge();\nStep 4. cool_apple_using_fridge();\nStep 5. go_to_the_diningtable();\nStep 6. put_cooled_apple_on_diningtable().", \
"heat": "Step 1. apple_name, location_name = locate_an_apple(candidate_locations);\nStep 2. take_an_apple(apple_name, location_name);\nStep 3. go_to_the_microwave();\nStep 4. heat_apple_using_microwave();\nStep 5. go_to_the_diningtable();\nStep 6. put_heated_apple_on_diningtable().", \
"examine": "Step 1. alarmclock_name, location_name = locate_an_alarmclock(candidate_locations);\nStep 2. take_an_alarmclock(alarmclock_name, location_name);\nStep 3. desklamp_name = find_an_desklamp(candidate_locations);\nStep 4. use_an_desklamp_to_examine(desklamp_name)."
}
task_ids = ['put','heat','cool','clean','examine']
results = []
with open('./data/pcode_plan_prompts/alfworld_plan_extraction_prompt.json', 'w') as f:
    for id in task_ids:
        icl_example_demo = icl_example.replace("<task>",example_task[id]).replace("<traj>",example_traj[id]).replace("<plan>",example_plan[id])
        print(icl_example_demo)
        result = {"task_id":id,"example_task":example_task[id],"example_traj":example_traj[id], "example_plan":example_plan[id],"icl_example":icl_example_demo}
        results.append(result)
    json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
icl_examples = {}
with open('./data/pcode_plan_prompts/alfworld_plan_extraction_prompt.json', encoding="utf-8") as f:
    data = json.load(f)
    for line in data:
        icl_examples[line['task_id']]=line['icl_example']

data_path = "./data/ReAct_style_data/alfworld_sft.json"
output_path = "./data/alfworld_sft_with_plan.json"
with open(output_path,'w',encoding='utf-8') as f:
    results = []
    with open(data_path, encoding="utf-8") as f:
        data=json.load(f)
        for line in data:
            id = line['id']
            game_file = line['game_file']
            convs = line['conversations']
            sys_prompt = "[Task Description]:\n" + convs[0]['value']
            task = "\n\n[Task]:\n" + convs[2]['value']
            task_id = getTaskType(task)
            icl_example = icl_examples[task_id]
            trajs = "\n\n[Solution Trajectory]:\n"
            for conv in convs[3:]:
                trajs += conv['value'] + "\n"
            icl_example = icl_example.replace("<sys_prompt>",sys_prompt)
            query = sys_prompt + task + trajs + "\n[Plan]:"
            input_prompt = extract_plan_prompt + icl_example + query
            response = llm._call(input_prompt)
            if response is not None:
                plan = "Thought: First, I devise a plan for solving the task:\n"+response
                line['conversations'][3]['value'] = plan + "\n" + line['conversations'][3]['value'][9:]
                results.append(line)
            else:
                print(f"error meets when id={id}")
    json.dump(results, f, ensure_ascii=False, indent=4)

### **WebShop**

In [ ]:
icl_example = "Here is one example.\n<sys_prompt>\n\n[Task]:\n<task>\n\n[Solution Trajectory]:\n<traj>\n\n[Plan]:\n<plan>\n\nNow is your turn.\n" 
example_task = "WebShop [SEP] Instruction: [SEP] Find me men's shorts with elastic waist, classic fit, short sleeve for gym workout with color: navy, and size: x-large, and price lower than 50.00 dollars [SEP] Search"
example_traj = "Thought: I think I should use the search bar to look for the shorts that match the given requirements.\nAction: search[men's shorts elastic waist classic fit short sleeve gym workout navy x-large price < 50.00]\nObservation: [SEP] Find me men's shorts with elastic waist, classic fit, short sleeve for gym workout with color: navy, and size: x-large, and price lower than 50.00 dollars [SEP] Back to Search [SEP] Page 1 (Total results: 50) [SEP] Next > [SEP] B09Q5ZHRVM [SEP] Yinimo Mens Gym Shorts Fashionable Loose Beach Drawstring Turkey Print Beach Cool Shorts [SEP] $19.19 [SEP] B09QCVCYVY [SEP] Burband Womens High Waist Bike Shorts Tummy Control Workout Yoga Pants Running Exercise Running Shorts with Side Pockets [SEP] $6.82 [SEP] B099231V35 [SEP] Mens Casual Cargo Pants Hi Vis Viz Reflective Overalls High Visibility Safe Work Pants Outdoor Hiking Trousers Big and Tall [SEP] $16.79 [SEP] B09QCP4579 [SEP] LWLW High Waist Yoga Shorts for Women,Tummy Control Biker Shorts Exercise Workout Butt Lifting Tights Women's Short Pants [SEP] $13.99 [SEP] B09S3BN15C [SEP] Mens Linen Shirt,Men's Striped Shirts Casual Short Sleeve Button Down Shirts Regular Fit Hawaiian Shirts Beach Tees Tops [SEP] $3.78 to $11.38 [SEP] B09R9YCM6R [SEP] XXBR Summer T-shirts for Mens, Soldier Short Sleeve 3D Street Vintage Printed Shirt Slim Fit Muscle Casual Tee Tops [SEP] $8.98 to $11.99 [SEP] B09Q67H373 [SEP] JSPOYOU 2022 Newly T-Shirt for Mens Funny 3D Graphics Pattern Crewneck Short Sleeve Tees Big and Tall Summer Casual Comfy Top [SEP] $1.99 to $8.99 [SEP] B09PYSKD7H [SEP] FIN86 Men's Elastic High Waisted Shorts,Men's Summer Printed Casual Shorts Loose Tether Pocket Board Shorts [SEP] $9.98 to $12.98 [SEP] B09HX5CD2D [SEP] CSU Cleveland State University Vikings Property Fleece Drawstring Shorts Heather Charcoal [SEP] $39.95 [SEP] B09HGFV91W [SEP] ZHDD Tops for Mens, Men's Knight Punk Vintage Style Long Sleeve Asymmetric Hem Cowl Neck Teen Boys Gothic Pullover [SEP] $4.98 to $7.99\nThought: I think I should click on the 'B09PYSKD7H' option as it seems to be men's shorts with elastic waist, classic fit, and possibly suitable for a gym workout.\nAction: click[B09PYSKD7H]\nObservation: [SEP] Find me men's shorts with elastic waist, classic fit, short sleeve for gym workout with color: navy, and size: x-large, and price lower than 50.00 dollars [SEP] Back to Search [SEP] < Prev [SEP] color [SEP] blue [SEP] navy [SEP] size [SEP] small [SEP] medium [SEP] large [SEP] x-large [SEP] xx-large [SEP] FIN86 Men's Elastic High Waisted Shorts,Men's Summer Printed Casual Shorts Loose Tether Pocket Board Shorts [SEP] Price: $9.98 to $12.98 [SEP] Rating: N.A. [SEP] Description [SEP] Features [SEP] Reviews [SEP] Buy Now\nThought: I think I should click on the 'navy' option to select the color I need.\nAction: click[navy]\nObservation: [SEP] Find me men's shorts with elastic waist, classic fit, short sleeve for gym workout with color: navy, and size: x-large, and price lower than 50.00 dollars [SEP] Back to Search [SEP] < Prev [SEP] color [SEP] blue [SEP] navy [SEP] size [SEP] small [SEP] medium [SEP] large [SEP] x-large [SEP] xx-large [SEP] FIN86 Men's Elastic High Waisted Shorts,Men's Summer Printed Casual Shorts Loose Tether Pocket Board Shorts [SEP] Price: $9.98 to $12.98 [SEP] Rating: N.A. [SEP] Description [SEP] Features [SEP] Reviews [SEP] Buy Now\nThought: I think I should click on the 'x-large' option to select the size I need.\nAction: click[x-large]\nObservation: [SEP] Find me men's shorts with elastic waist, classic fit, short sleeve for gym workout with color: navy, and size: x-large, and price lower than 50.00 dollars [SEP] Back to Search [SEP] < Prev [SEP] color [SEP] blue [SEP] navy [SEP] size [SEP] small [SEP] medium [SEP] large [SEP] x-large [SEP] xx-large [SEP] FIN86 Men's Elastic High Waisted Shorts,Men's Summer Printed Casual Shorts Loose Tether Pocket Board Shorts [SEP] Price: $9.98 to $12.98 [SEP] Rating: N.A. [SEP] Description [SEP] Features [SEP] Reviews [SEP] Buy Now\nThought: I think I have found the right product that matches all the requirements. It's time to buy it.\nAction: click[Buy Now]"
example_plan = "Step 1. selected_item = search_item(\"men's shorts with elastic waist, classic fit, short sleeve for gym workout with color: navy, and size: x-large, and price lower than 50.00 dollars\");\nStep 2. click(selected_item);\nStep 3. select_color(\"navy\");\nStep 4. select_size(\"x-large\");\nStep 5. buy()."
# example_plan = ""
icl_example = icl_example.replace("<task>",example_task).replace("<traj>",example_traj).replace("<plan>",example_plan)
print(icl_example)
results = []
with open('./data/pcode_plan_prompts/webshop_plan_extraction_prompt.json', 'w') as f:
    result = {"example_task":example_task,"example_traj":example_traj, "example_plan":example_plan,"icl_example":icl_example}
    results.append(result)
    json.dump(results, f, ensure_ascii=False, indent=4)

In [10]:
with open('./data/pcode_plan_prompts/webshop_plan_extraction_prompt.json', encoding="utf-8") as f:
    data = json.load(f)[0]
    icl_example=data['icl_example']

data_path = "./data/ReAct_style_data/webshop_sft.json"
output_path = "./data/webshop_sft_with_plan.json"
data_to_upload = []
with open(output_path,'w',encoding='utf-8') as f:
    results = []
    with open(data_path, encoding="utf-8") as f:
        data=json.load(f)
        for line in data:
            id = line['id']
            convs = line['conversations']
            sys_prompt = "[Task Description]:\n" + convs[0]['value']
            task = "\n\n[Task]:\n" + convs[2]['value']
            trajs = "\n\n[Solution Trajectory]:\n"
            for conv in convs[3:]:
                trajs += conv['value'] + "\n"
            icl_example = icl_example.replace("<sys_prompt>",sys_prompt)
            query = sys_prompt + task + trajs + "\n[Plan]:"
            input_prompt = extract_plan_prompt + icl_example + query
            # print(input_prompt)
            data_to_upload.append([id, input_prompt])
            response = llm._call(input_prompt)
            if response is not None:
                plan = "Thought: First, I devise a plan for solving the task:\n"+response
                line['conversations'][3]['value'] = plan + "\n" + line['conversations'][3]['value'][9:]
                results.append(line)
            else:
                print(f"error meets when id={id}")
    json.dump(results, f, ensure_ascii=False, indent=4)

### **TextCraft**

In [ ]:
icl_example = None
with open('./data/pcode_plan_prompts/textcraft_plan_extraction_prompt.json', encoding="utf-8") as f:
    data = json.load(f)
    for line in data:
        icl_example=line['icl_example']

output_path = "./data/textcraft_sft_with_plan.json"
data_path = "./data/ReAct_style_data/textcraft_sft.json"
with open(output_path,'w',encoding='utf-8') as f:
    results = []
    with open(data_path, encoding="utf-8") as f:
        data=json.load(f)
        for line in data:
            id = line['item_id'].split('_')[-1].strip()
            convs = line['conversations']
            sys_prompt = "[Task Description]:\n" + convs[0]['value']
            task = "\n\n[Task]:\n" + convs[2]['value'].split("Observation: ")[-1].strip()
            trajs = "\n\n[Solution Trajectory]:\n"
            for conv in convs[3:]:
                trajs += conv['value'] + "\n"
            icl_example = icl_example.replace("<sys_prompt>",sys_prompt)
            query = sys_prompt + task + trajs + "\n[Plan]:"
            input_prompt = extract_plan_prompt + icl_example + query
            response = llm._call(input_prompt)
            if response is not None:
                plan = "Thought: First, I devise a plan for solving the task:\n"+response
                line['conversations'][3]['value'] = plan + "\n" + line['conversations'][3]['value'][9:]
                results.append(line)
            else:
                print(f"error meets when id={id}")
    json.dump(results, f, ensure_ascii=False, indent=4)

### **SciWorld**

In [ ]:
def getTaskType(task):
    if "Your task is to boil" in task:
        return '1-1'
    elif "Your task is to melt" in task:
        return '1-2'
    elif "Your task is to freeze" in task: 
        return '1-3'
    elif "Your task is to change the state of matter of" in task:
        return '1-4'
    elif "Your task is to measure the melting point of" in task:
        return '2-2'
    elif "Your task is to measure the temperature of" in task:
        return '2-1'
    elif "Your task is to turn on" in task and "by powering it" in task:
        return '3-2'
    elif "Your task is to turn on" in task:
        return '3-1'
    elif "Your task is to determine if unknown substance" in task and "is electrically conductive. " in task:
        return '3-4'
    elif "is electrically conductive. " in task: 
        return '3-3'
    elif "Your task is to find a(n) animal." in task:
        return '4-4'
    elif "Your task is to find a(n) living thing." in task:
        return '4-1'
    elif "Your task is to find a(n) non-living thing." in task:
        return '4-2'
    elif "Your task is to find a(n) plant." in task:
        return '4-3'
    elif "Your task is to grow " in task and "plant" in task: # grow-plant
        return '5-1'
    elif "Your task is to grow " in task: # grow-fruit
        return '5-2'
    elif "Your task is to use chemistry to create the substance" in task:
        return '6-1'
    elif "Your task is to use chemistry to create" in task and "-" in task:
        return '6-2'
    elif "Your task is to use chemistry " in task: 
        return '6-3'
    elif "find the animal with the longest life span." in task:
        return '7-1'
    elif "find the animal with the shortest life span." in task:
        return '7-2'
    elif "find the animal with the longest life span, then the shortest life span." in task:
        return '7-3'
    elif "Your task is to focus on the life stages of" in task:
        return '8-2'
    elif "Your task is to focus" in task:
        return '8-1'
    else:
        print('loss')

In [ ]:
icl_example = "Here is one example.\n<sys_prompt>\n\n[Task]:\n<task>\n\n[Solution Trajectory]:\n<traj>\n\n[Plan]:\n<plan>\n\nNow is your turn.\n" 
example_task = {"1-1": "Your task is to boil orange juice. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.", \
"1-2": "", \
"1-3": "Your task is to freeze chocolate. First, focus on the substance. Then, take actions that will cause it to change its state of matter.", \
'1-4': "Your task is to change the state of matter of chocolate. First, focus on the substance. Then, take actions that will cause it to change its state of matter.", \
'2-1': "Your task is to measure the temperature of unknown substance B, which is located around the living room. First, focus on the thermometer. Next, focus on the unknown substance B. If the unknown substance B temperature is above 100.0 degrees, place it in the red box. If the unknown substance B temperature is below 100.0 degrees, place it in the green box. The boxes are located around the bathroom.", \
'2-2': "Your task is to measure the melting point of chocolate, which is located around the kitchen. First, focus on the thermometer. Next, focus on the chocolate. If the melting point of chocolate is above -10.0 degrees, focus on the blue box. If the melting point of chocolate is below -10.0 degrees, focus on the orange box. The boxes are located around the kitchen.", \
'3-1': "", \
'3-2': "Your task is to turn on the red light bulb by powering it using a nonrenewable power source. First, focus on the red light bulb. Then, create an electrical circuit that powers it on.", \
'3-3': "", \
'3-4': "Your task is to determine if unknown substance J is electrically conductive. The unknown substance J is located around the workshop. First, focus on the unknown substance J. If it is electrically conductive, place it in the red box. If it is electrically nonconductive, place it in the green box.", \
'4-1': "", \
'4-2': "", \
'4-3': "", \
'4-4': "", \
'5-1': "", \
'5-2': "", \
'6-1': "Your task is to use chemistry to create the substance 'soapy water'. A recipe and some of the ingredients might be found near the kitchen. When you are done, focus on the soapy water.", \
'6-2': "Your task is to use chemistry to create green-blue paint. When you are part-way done, focus on the intermediate (secondary color) paint you created.  When you are completely done, focus on the green paint.", \
'6-3': "Your task is to use chemistry to create green paint. When you are done, focus on the green paint.", \
'7-1': "Your task is to find the animal with the longest life span.  The animals are in the 'outside' location.  Focus on the animal with the longest life span.", \
'7-2': "", \
'7-3': "", \
'8-1': "", \
'8-2': "Your task is to focus on the 4 life stages of the turtle, starting from earliest to latest."
}
example_traj = {"1-1": "Thought: I need to find the orange juice to complete the task of boiling it. I should look around to identify my surroundings and find the kitchen where the orange juice is likely to be.\nAction: look around\nObservation: This room is called the hallway. In it, you see: \n\tthe agent\n\ta drawing\n\ta substance called air\nYou also see:\n\tA door to the greenhouse (that is open)\n\tA door to the living room (that is open)\n\tA door to the art studio (that is open)\n\tA door to the kitchen (that is open)\n\tA door to the bedroom (that is open)\n\tA door to the workshop (that is open)\nThought: The hallway does not contain the items necessary for boiling orange juice. I should teleport to the kitchen where I can find the required tools and the orange juice.\nAction: teleport to kitchenObservation: You teleport to the kitchen.\nThought: Now that I am in the kitchen, I should look around to locate the equipment or tools needed to boil the orange juice.\nAction: look around\nObservation: This room is called the kitchen. In it, you see: \n\ta stopwatch, which is deactivated.\n\ta oven, which is turned off. The oven door is closed.\n\ta thermometer, currently reading a temperature of 10 degrees celsius\n\ta counter. On the counter is: a bowl (containing a banana, a red apple, an orange, a potato), a drawer.\n\ta stove, which is turned off. On the stove is: nothing.\n\ta table. On the table is: a glass cup (containing nothing).\n\ta sink, which is turned off. In the sink is: nothing.\n\ta freezer. The freezer door is closed.\n\ta cupboard. The cupboard door is closed.\n\ta glass jar (containing a substance called sodium chloride)\n\ta lighter\n\tthe agent\n\ta substance called air\n\ta fridge. The fridge door is closed.\n\ta chair. On the chair is: nothing.\n\ta painting\n\ta substance called soap\n\tYou also see:\n\tA door to the hallway (that is open)\n\tA door to the outside (that is open)\n\tA door to the bathroom (that is open)\nThought: I chose to pick up the metal pot because it is likely needed to hold the orange juice for boiling on the stove.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I chose to open the fridge to look for orange juice to utilize for the task of boiling orange juice.\nAction: open fridge\nObservation: The fridge is now open. In the fridge is: a wood cup (containing orange juice).\nThought: I chose to focus on the orange juice because it is necessary to identify and interact with the primary substance required for my task, which is to boil orange juice.\nAction: focus on orange juice\nObservation: You focus on the orange juice.\nThought: I chose to pour the orange juice into the metal pot because it's necessary to transfer the juice into a cookware item that can be used on the stove for heating.\nAction: pour cup containing orange juice into metal pot\nObservation: You pour the contents of the wood cup into the metal pot.\nThought: I'm picking up the metal pot to move it to a heat source, like the stove, to start boiling the orange juice.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I need to move the metal pot to the stove to heat the orange juice as part of my task to change its state of matter by boiling it.\nAction: move metal pot to stove\nObservation: You move the metal pot to the stove.\nThought: Activating the stove will provide the heat needed to attempt boiling or combusting the orange juice in the metal pot.\nAction: activate stove\nObservation: The stove appears broken, and can't be activated or deactivated.\nThought: I chose to pick up the metal pot to explore alternatives for heating the orange juice since the stove is broken.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I chose to move to the foundry, since blast furnace in the foundry may be suited for boiling orange juice.\nAction: teleport to foundry\nObservation: You move to the foundry.\nThought: I need open the blast furnace to prepare it for heating up the orange juice.\nAction: open blast furnace\nObservation: The blast furnace is now open.\nThought: I should move the metal pot to the blast furnace to prepare for heating the orange juice, as heat is required to change its state of matter.\nAction: move metal pot to blast furnace\nObservation: You move the metal pot to the blast furnace.\nThought: I chose to activate the blast furnace to heat the orange juice in the metal pot, aiming to boil it or combust it, as required by the task.\nAction: activate blast furnace\nObservation: The blast furnace is now activated.\nThought: I chose to monitor the blast furnace to examine the current state of orange juice, given the task of boiling it.\nAction: look at blast furnace\nObservation: a blast furnace, which is turned on. The blast furnace door is open. In the blast furnace is: \n\ta metal pot (containing orange juice)\nThought: It's important to keep checking the blast furnace to ensure the orange juice changes its state of matter and to prevent any accidents.\nAction: look at blast furnace", \
"1-2": "", \
"1-3": "Thought: I need to locate the chocolate to freeze it. The first step is to survey my current environment to identify if I'm in the right place to find chocolate.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta wood cup (containing blue paint)\n\ta substance called air\n\ta wood cup (containing red paint)\n\tthe agent\n\ta wood cup (containing yellow paint)\n\ta large cupboard. The large cupboard door is closed. \n\ta table. On the table is: a bowl (containing nothing).\nYou also see:\n\tA door to the hallway (that is open)\nThought: The art studio doesn't have the items I need to freeze chocolate. I should go to the kitchen where I'm more likely to find chocolate and freezing equipment.\nAction: teleport to kitchenObservation: You move to the kitchen.\nThought: I chose to look around to gather information about my surroundings and locate the chocolate or freezing equipment needed to complete my task.\nAction: look around\nObservation: This room is called the kitchen. In it, you see: \n\ta chair. On the chair is: nothing.\n\ta freezer. The freezer door is closed. \n\ta sink, which is turned off. In the sink is: nothing.\n\ta cupboard. The cupboard door is closed. \n\ta glass jar (containing a substance called sodium chloride)\n\ta stopwatch, which is deactivated. \n\tthe agent\n\ta oven, which is turned off. The oven door is closed. \n\ta substance called air\n\ta thermometer, currently reading a temperature of 10 degrees celsius\n\ta lighter\n\ta fridge. The fridge door is closed. \n\ta table. On the table is: a glass cup (containing nothing).\n\ta painting\n\ta stove, which is turned off. On the stove is: nothing.\n\ta counter. On the counter is: a drawer, a bowl (containing an orange, a potato, a red apple, a banana).\n\ta substance called soap\nYou also see:\n\tA door to the hallway (that is open)\n\tA door to the bathroom (that is open)\n\tA door to the outside (that is open)\nThought: I chose to open the cupboard to search for chocolate or items that might help in freezing it.\nAction: open cupboard\nObservation: The cupboard is now open. In the cupboard is: a ceramic cup (containing nothing), a drawer, a metal pot (containing nothing), a tin cup (containing nothing).\nThought: I can pick up the metal pot to potentially use it in a step for freezing chocolate, possibly involving a heat transfer or containment mechanism.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I choose to open the fridge to look for chocolate inside and proceed with freezing it.\nAction: open fridge\nObservation: The fridge door is open. In the fridge is: chocolate, a wood cup (containing orange juice).\nThought: I should focus on the chocolate to identify it because it is the substance I need to modify in order to complete my task of freezing it.\nAction: focus on chocolate\nObservation: You focus on the chocolate.\nThought: I need place the chocolate in the metal pot in preparation to manipulate its environment, specifically to change its temperature.\nAction: move chocolate to metal pot\nObservation: You move the chocolate to the metal pot.\nThought: I should pick up the metal pot containing the chocolate to transport it to a location where I can freeze it.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I need place the metal pot containing chocolate on the stove to heat it before freezing, as part of the process to change its state of matter.\nAction: move metal pot to stove\nObservation: You move the metal pot to the stove.\nThought: I am activating the stove to heat the chocolate in the metal pot, leading it to melt before I proceed to freeze it.\nAction: activate stove\nObservation: The stove is now activated.\nThought: I want to check the current state of the chocolate to determine if the heat from the stove has started to melt it, which is essential before I can take steps to freeze it.\nAction: look at stove\nObservation: a stove, which is turned on. On the stove is: a metal pot (containing chocolate).\nThought: Continuing to monitor the chocolate to ensure it's melting properly.\nAction: look at stove\nObservation: a stove, which is turned on. On the stove is: a metal pot (containing chocolate).\nThought: I'm still waiting for the chocolate to melt completely before proceeding to the next step.\nAction: look at stove\nObservation: a stove, which is turned on. On the stove is: a metal pot (containing chocolate).\nThought: I need to keep an eye on the chocolate's melting process, watching for any changes.\nAction: look at stove\nObservation: a stove, which is turned on. On the stove is: a metal pot (containing liquid chocolate).\nThought: Now that the chocolate is liquid, I should move it from the stove to begin the freezing process.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I want to open the freezer to create the right conditions to freeze the chocolate.\nAction: open freezer\nObservation: The freezer is now open.\nThought: I decided to place the metal pot containing liquid chocolate in the freezer to initiate the process of changing its state from liquid to solid by freezing.\nAction: move metal pot to freezer\nObservation: You move the metal pot to the freezer.\nThought: I need examine the chocolate to check its current state and see if it has started to freeze yet.\nAction: look at freezer\nObservation: The freezer door is open. In the freezer is: a metal pot (containing liquid chocolate).\nThought: I chose to wait to see if the state of the chocolate changes.\nAction: wait\nObservation: You decide to wait for 10 iterations.\nThought: I chose to examine the chocolate to assess its current state and determine how close it is to freezing.\nAction: look at freezer\nObservation: The freezer door is open. In the freezer is: a metal pot (containing liquid chocolate).\nThought: The chocolate still needs more time to freeze. I will continue to wait and check back later.\nAction: wait\nObservation: You decide to wait for 10 iterations.\nThought: I chose to examine the chocolate to assess its current state and determine how much closer it is to freezing.\nAction: look at freezer\nObservation: The freezer door is open. In the freezer is: a metal pot (containing solid chocolate).\nThought: Now, the chocolate has frozen. I chose to wait to observe if the state of the chocolate changes further that could influence my next steps in freezing the chocolate.\nAction: wait", \
'1-4': "Thought: To change the state of matter of chocolate, I need to find the chocolate and determine the necessary conditions to alter its state.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta table. On the table is: a bowl (containing nothing).\n\ta substance called air\n\ta large cupboard. The large cupboard door is closed. \n\ta wood cup (containing blue paint)\n\ta wood cup (containing red paint)\n\tthe agent\n\ta wood cup (containing yellow paint)\nYou also see:\n\tA door to the hallway (that is open)\nThought: I need to go to the kitchen where I might find items to help change the state of chocolate, such as heat or cooling elements.\nAction: teleport to kitchenObservation: You teleport to the kitchen.\nThought: I need look around the kitchen to find chocolate or tools for changing its state of matter.\nAction: look around\nObservation: This room is called the kitchen. In it, you see: \n\ta glass jar (containing a substance called sodium chloride)\n\ta sink, which is turned off. In the sink is: nothing.\n\ta stove, which is turned off. On the stove is: nothing.\n\ta painting\n\ta table. On the table is: a glass cup (containing nothing).\n\ta lighter\n\ta substance called air\n\tthe agent\n\ta cupboard. The cupboard door is closed. \n\ta oven, which is turned off. The oven door is closed. \n\ta thermometer, currently reading a temperature of 10 degrees celsius\n\ta freezer. The freezer door is closed. \n\ta fridge. The fridge door is closed. \n\ta chair. On the chair is: nothing.\n\ta stopwatch, which is deactivated. \n\ta counter. On the counter is: a drawer, a bowl (containing a red apple, a potato, a banana, an orange).\n\ta substance called soap\nYou also see:\n\tA door to the hallway (that is open)\n\tA door to the outside (that is open)\n\tA door to the bathroom (that is open)\nThought: I'm opening the cupboard to search for items which help complete my task of changing the state of matter of chocolate.\nAction: open cupboard\nObservation: The cupboard is now open. In the cupboard is: a ceramic cup (containing nothing), a drawer, a metal pot (containing nothing), a tin cup (containing nothing).\nThought: I need pick up the metal pot to use it on the stove or oven for heating the chocolate to change its state by melting.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I am opening the fridge to explore its contents, potentially finding the chocolate or suitable conditions for changing its state of matter.\nAction: open fridge\nObservation: The fridge is now open. In the fridge is: chocolate, a wood cup (containing orange juice).\nThought: I chose to focus on the chocolate because it's essential to clearly identify the substance I need to change the state of, as per the task description.\nAction: focus on chocolate\nObservation: You focus on the chocolate.\nThought: I need transfer the chocolate to the metal pot to use it as a container for changing its state by applying heat or cold.\nAction: move chocolate to metal pot\nObservation: You move the chocolate to the metal pot.\nThought: I need pick up the metal pot containing chocolate to prepare for altering its state of matter through heating.\nAction: pick up metal pot\nObservation: You move the metal pot to the inventory.\nThought: I should move the metal pot to the stove to use heat to change the chocolate from a solid to a liquid state.\nAction: move metal pot to stove\nObservation: You move the metal pot to the stove.\nThought: I need turn on the stove to heat the chocolate in the pot, which should melt it and change its state from solid to liquid.\nAction: activate stove\nObservation: The stove is now activated.\nThought: I want to check the current state of the chocolate to see if it has started melting from the heat of the stove.\nAction: look at stove\nObservation: a stove, which is turned on. On the stove is: a metal pot (containing chocolate).\nThought: I am still examining the chocolate to check its current state and see if it's begun melting so I can properly adjust the next steps.\nAction: look at stove", \
'2-1': "Thought: I need to locate the thermometer to measure the temperature of unknown substance B. First, I should look around the current room to identify my surroundings.\nAction: look around\nObservation: This room is called the hallway. In it, you see: \n\tthe agent\n\ta substance called air\n\ta picture\nYou also see:\n\tA door to the green house (that is open)\n\tA door to the living room (that is open)\n\tA door to the art studio (that is open)\n\tA door to the kitchen (that is open)\n\tA door to the bedroom (that is open)\n\tA door to the workshop (that is open)\n[Solution Trajectory]\nThought: The hallway doesn't have the items I need. I should teleport to the kitchen, as that's a likely place to find a thermometer.\nAction: teleport to kitchenObservation: You teleport to the kitchen.\nThought: I chose to look around to gather information about the environment and locate the thermometer necessary for measuring the temperature of substance B.\nAction: look around\nObservation: This room is called the kitchen. In it, you see: \n\ta oven, which is turned off. The oven door is closed. \n\ta cupboard. The cupboard door is closed. \n\tthe agent\n\ta fridge. The fridge door is closed. \n\ta glass jar (containing a substance called sodium chloride)\n\ta lighter\n\ta stopwatch, which is deactivated. \n\ta substance called soap\n\ta table. On the table is: a glass cup (containing nothing).\n\ta painting\n\ta freezer. The freezer door is closed. \n\ta thermometer, currently reading a temperature of 10 degrees celsius\n\ta substance called air\n\ta chair. On the chair is: nothing.\n\ta sink, which is turned off. In the sink is: nothing.\n\ta stove, which is turned off. On the stove is: nothing.\n\ta counter. On the counter is: a drawer, a bowl (containing a banana, an orange, a potato, a red apple).\nYou also see:\n\tA door to the hallway (that is open)\n\tA door to the bathroom (that is open)\n\tA door to the outside (that is open)\nThought: I chose to pick up the thermometer to measure the temperature of the unknown substance B as required in the task description.\nAction: pick up thermometer\nObservation: You move the thermometer to the inventory.\nThought: I want to focus on the thermometer in my inventory to prepare it for measuring the temperature of the unknown substance B.\nAction: focus on thermometer in inventory\nObservation: You focus on the thermometer.\nThought: I chose to go to the living room where the unknown substance B is located.\nAction: teleport to living room\nObservation: You teleport to the living room.\nThought: I should look around to locate the unknown substance B.\nAction: look around\nObservation: This room is called the living room. In it, you see: \n\ta couch. On the couch is: a white pillow.\n\tunknown substance B\n\ta substance called air\n\ta chair. On the chair is: nothing.\n\tthe agent\n\ta desk. On the desk is: a drawer.\n\ta painting\nYou also see:\n\tA door to the hallway (that is open)\nThought: There's unknown substance B. I should pick up unknown substance B to measure its temperature as required by the task.\nAction: pick up unknown substance B\nObservation: You move the unknown substance B to the inventory.\nThought: My action is to focus on the unknown substance B in my inventory because it is necessary to measure its temperature next, as specified in the task description.\nAction: focus on unknown substance B in inventory\nObservation: You focus on the unknown substance B.\nThought: I should move towards the bathroom where I need to place the unknown substance B based on its temperature.\nAction: teleport to bathroom\nObservation: You teleport to the bathroom.\nThought: I need to identify where the red and green boxes are located to complete my task effectively after I measure the temperature of substance B.\nAction: look around\nObservation: This room is called the bathroom. In it, you see: \n\tthe agent\n\ta substance called air\n\ta red box (containing nothing)\n\ta green box (containing nothing)\n\ta drawing\n\ta bath tub, which is turned off. In the bath tub is: nothing.\n\ta sink, which is turned off. In the sink is: nothing.\n\ta toilet. In the toilet is: A drain, which is closed, a substance called water.\n\ta glass cup (containing nothing)\nYou also see:\n\tA door to the kitchen (that is open)\nThought: I need to measure the temperature of unknown substance B to determine which box it should be placed in.\nAction: use thermometer in inventory on unknown substance B in inventory\nObservation: the thermometer measures a temperature of 29 degrees celsius\nThought: Given the temperature of unknown substance B is below 100.0 degrees celsius, placing it in the green box is appropriate according to the task instructions.\nAction: move unknown substance B in inventory to green box", \
'2-2': "Thought: I need to find the kitchen where the chocolate and thermometer are likely located.\nAction: look around\nObservation: This room is called the hallway. In it, you see: \n\ta picture\n\tthe agent\n\ta substance called air\nYou also see:\n\tA door to the green house (that is open)\n\tA door to the living room (that is open)\n\tA door to the art studio (that is open)\n\tA door to the kitchen (that is open)\n\tA door to the bedroom (that is open)\n\tA door to the workshop (that is open)\nThought: The living room doesn't have the items I need. I should go to the kitchen.\nAction: teleport to kitchenObservation: You teleport to the kitchen.\nThought: I chose to look around to gather information about my surroundings and locate the thermometer and chocolate needed for the task.\nAction: look around\nObservation: This room is called the kitchen. In it, you see: \n\ta blue box (containing nothing)\n\ta sink, which is turned off. In the sink is: nothing.\n\ta stove, which is turned off. On the stove is: nothing.\n\ta painting\n\ta oven, which is turned off. The oven door is closed. \n\ta table. On the table is: a glass cup (containing nothing).\n\ta freezer. The freezer door is closed. \n\ta stopwatch, which is deactivated. \n\ta thermometer, currently reading a temperature of 10 degrees celsius\n\ta lighter\n\ta fridge. The fridge door is closed. \n\ta substance called soap\n\ta glass jar (containing a substance called sodium chloride)\n\tthe agent\n\ta counter. On the counter is: a bowl (containing a potato, an orange, a red apple, a banana), a drawer.\n\ta orange box (containing nothing)\n\ta substance called air\n\ta chair. On the chair is: nothing.\n\ta cupboard. The cupboard door is closed. \nYou also see:\n\tA door to the bathroom (that is open)\n\tA door to the outside (that is open)\n\tA door to the hallway (that is open)\nThought: I chose to pick up the thermometer because it is necessary to measure the temperature of the chocolate, which is the first step in determining its melting point.\nAction: pick up thermometer\nObservation: You move the thermometer to the inventory.\nThought: By focusing on the thermometer in my inventory, I can closely observe its readings and ensure accuracy before using it to measure the melting point of the chocolate.\nAction: focus on thermometer in inventory\nObservation: You focus on the thermometer.\nThought: I am choosing to \"look around\" to gather more information about the kitchen and locate the chocolate needed for the task.\nAction: look around\nObservation: This room is called the kitchen. In it, you see: \n\ta blue box (containing nothing)\n\ta sink, which is turned off. In the sink is: nothing.\n\ta stove, which is turned off. On the stove is: nothing.\n\ta painting\n\ta oven, which is turned off. The oven door is closed. \n\ta table. On the table is: a glass cup (containing nothing).\n\ta freezer. The freezer door is closed. \n\ta stopwatch, which is deactivated. \n\ta lighter\n\ta fridge. The fridge door is closed. \n\ta substance called soap\n\ta glass jar (containing a substance called sodium chloride)\n\tthe agent\n\ta counter. On the counter is: a bowl (containing a potato, an orange, a red apple, a banana), a drawer.\n\ta orange box (containing nothing)\n\ta substance called air\n\ta chair. On the chair is: nothing.\n\ta cupboard. The cupboard door is closed. \nYou also see:\n\tA door to the bathroom (that is open)\n\tA door to the outside (that is open)\n\tA door to the hallway (that is open)\nThought: I chose to open the fridge to look for the chocolate as it is a key item required to measure its melting point.\nAction: open fridge\nObservation: The fridge is now open. In the fridge is: chocolate, a wood cup (containing orange juice).\nThought: I chose to focus on the chocolate to continue following the steps in the task description and eventually measure its melting point.\nAction: focus on chocolate\nObservation: You focus on the chocolate.\nThought: I open the cupboard to see if there are any other items or tools that could assist me in measuring the melting point of the chocolate.\nAction: open cupboard\nObservation: The cupboard is already open. In the cupboard is: a ceramic cup (containing nothing), a drawer, a metal pot (containing nothing), a tin cup (containing nothing).\nThought: I can move the chocolate to the metal pot to prepare for measuring its melting point using a heat source like the stove.\nAction: move chocolate to metal pot\nObservation: You move the chocolate to the metal pot.\nThought: I should move the metal pot with the chocolate to the stove to heat it and measure its melting point using the thermometer.\nAction: move metal pot to stove\nObservation: You move the metal pot to the stove.\nThought: I chose to activate the stove to heat the metal pot containing chocolate, as this is necessary to observe the melting point of the chocolate.\nAction: activate stove\nObservation: The stove is now activated.\nThought: I want to examine the chocolate to determine its current state and see if it's beginning to melt, which is essential for checking whether it's reaching its melting point.\nAction: examine chocolate\nObservation: chocolate\nThought: I'm using the thermometer to measure the chocolate's temperature to determine its melting point.\nAction: use thermometer in inventory on chocolate\nObservation: the thermometer measures a temperature of 9 degrees celsius\nThought: I chose to examine the chocolate to verify its current state and properties, ensuring the accuracy of the temperature reading related to its melting point.\nAction: examine chocolate\nObservation: chocolate\nThought: I chose to use the thermometer on the chocolate to accurately measure its temperature, which is essential for determining its melting point.\nAction: use thermometer in inventory on chocolate\nObservation: the thermometer measures a temperature of 15 degrees celsius\nThought: I'm examining the chocolate to confirm its state before verifying if its melting point is above or below -10.0 degrees Celsius, as the task instructs.\nAction: examine chocolate\nObservation: chocolate\nThought: Using the thermometer on the chocolate will help measure its melting point, which is essential for deciding the next steps in the task.\nAction: use thermometer in inventory on chocolate\nObservation: the thermometer measures a temperature of 24 degrees celsius\nThought: I want to confirm the temperature of the chocolate directly before making a decision regarding the boxes.\nAction: examine chocolate\nObservation: liquid chocolate\nThought: I should use the thermometer on the chocolate to measure its melting point, as it's necessary for determining the correct box to focus on next.\nAction: use thermometer in inventory on chocolate\nObservation: the thermometer measures a temperature of 34 degrees celsius\nThought: I chose to focus on the blue box because the melting point of the chocolate measured above -10.0 degrees Celsius.\nAction: focus on blue box", \
'3-1': "", \
'3-2': "Thought: To turn on the red light bulb, I need to find it first. I should look around the current room to identify if the red light bulb is here or if I need to move to another location where it might be located.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta substance called air\n\ta table. On the table is: a bowl (containing nothing).\n\ta wood cup (containing yellow paint)\n\ta large cupboard. The large cupboard door is closed. \n\tthe agent\n\ta wood cup (containing red paint)\n\ta wood cup (containing blue paint)\nYou also see:\n\tA door to the hallway (that is open)\nThought: The red light bulb is not in the art studio. I can go to the workshop because it is likely to have the tools and materials needed to create an electrical circuit for the red light bulb.\nAction: teleport to workshop\nObservation: You teleport to the workshop.\nThought: I want to identify if there are any items in the workshop I can use to build an electrical circuit to power the red light bulb.\nAction: look around\nObservation: This room is called the workshop. In it, you see: \n\ta substance called air\n\ta electric motor, which is off\n\ta table. On the table is: a red light bulb, which is off, a battery, a blue light bulb, which is off, a green light bulb, which is off, a yellow wire, a switch, which is off, a red wire, a black wire.\n\ta ultra low temperature freezer. The ultra low temperature freezer door is closed. \n\ta electric buzzer, which is off\n\tthe agent\n\ta solar panel, which is off\n\ta nuclear generator, which is off\nYou also see:\n\tA door to the hallway (that is open)\nThought: I chose to focus on the red light bulb as it is the primary object I need to interact with to accomplish my task.\nAction: focus on red light bulb\nObservation: You focus on the red light bulb.\nThought: I want to gather more details about the surroundings and items available in the workshop to efficiently plan the creation of an electrical circuit.\nAction: look around\nObservation: This room is called the workshop. In it, you see: \n\ta substance called air\n\ta electric motor, which is off\n\ta table. On the table is: a red light bulb, which is off, a battery, a blue light bulb, which is off, a green light bulb, which is off, a yellow wire, a switch, which is off, a red wire, a black wire.\n\ta ultra low temperature freezer. The ultra low temperature freezer door is closed. \n\ta electric buzzer, which is off\n\tthe agent\n\ta solar panel, which is off\n\ta nuclear generator, which is off\nYou also see:\n\tA door to the hallway (that is open)\nThought: I can connect the nuclear generator anode to the yellow wire terminal 1 because I need to create a circuit powered by a nonrenewable source, and the nuclear generator is a suitable nonrenewable power source for this task.\nAction: connect nuclear generator anode to yellow wire terminal 1\nObservation: anode on nuclear generator is now connected to terminal 1 on yellow wire\nThought: I continue connecting the cathode of the nuclear generator to complete the power source circuit. I will use the red wire for this connection.\nAction: connect nuclear generator cathode to red wire terminal 1\nObservation: cathode on nuclear generator is now connected to terminal 1 on red wire\nThought: I chose to connect the yellow wire to the cathode in the red light bulb to complete the circuit essential for turning on the light bulb.\nAction: connect yellow wire terminal 2 to cathode in red light bulb\nObservation: terminal 2 on yellow wire is now connected to cathode on red light bulb\nThought: I need connect the red wire to the anode of the red light bulb to continue constructing the necessary electrical circuit needed to power it.\nAction: connect red wire terminal 2 to anode in red light bulb\nObservation: terminal 2 on red wire is now connected to anode on red light bulb\nThought: I should activate the nuclear generator to provide a nonrenewable power source for the red light bulb.\nAction: activate nuclear generator\nObservation: The nuclear generator is now activated.\nThought: I chose to wait because I need time to observe if activating the nuclear generator has any immediate effects on the environment or the red light bulb turns on as expected.\nAction: wait1.", \
'3-3': "", \
'3-4': "Thought: To determine if the unknown substance J is electrically conductive, I need to locate it first. It is said to be around the workshop, so I should start by looking around the current location to identify my surroundings and find a way to the workshop.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta wood cup (containing red paint)\n\ta wood cup (containing blue paint)\n\ta substance called air\n\ta table. On the table is: a glass cup (containing nothing).\n\ta wood cup (containing yellow paint)\n\tthe agent\n\ta large cupboard. The large cupboard door is closed. \nYou also see:\n\tA door to the hallway (that is open).Thought:\nI chose to teleport to the workshop to locate the unknown substance J, as it is essential for determining its electrical conductivity as per the task description.\nAction: teleport to workshop\nObservation: You teleport to the workshop.\nThought: I should pick up the unknown substance because it is the key item I need to test for electrical conductivity, as described in my task.\nAction: pick up unknown substance\nObservation: You move the unknown substance J to the inventory.\nThought: I need focus on the unknown substance J to analyze its properties and figure out if it's electrically conductive.\nAction: focus on unknown substance\nObservation: You focus on the unknown substance J.\nThought: I need drop the unknown substance J on the table so I can use the available apparatus to test its electrical conductivity.\nAction: drop unknown substance\nObservation: You move the unknown substance J to the workshop.\nThought: By looking around, I aim to acquire a comprehensive view of the workshop to locate and utilize tools or devices that can help me test the electrical conductivity of the unknown substance J.\nAction: look around\nObservation: This room is called the workshop. In it, you see: \n\ta table. On the table is: a yellow light bulb, which is off, a switch, which is off, a red light bulb, which is off, a green wire, a yellow wire, a battery, a blue wire, a blue light bulb, which is off.\n\ta green box (containing nothing)\n\ta ultra low temperature freezer. The ultra low temperature freezer door is closed. \n\ta substance called air\n\ta red box (containing nothing)\n\tunknown substance J\n\tthe agent\nYou also see:\n\tA door to the hallway (that is open)\nThought: I need connect the battery to the blue wire to start assembling a circuit which will help test the conductivity of unknown substance J.\nAction: connect battery anode to blue wire terminal 1\nObservation: anode on battery is now connected to terminal 1 on blue wire\nThought: Next, I'll connect the battery cathode to complete the circuit and start constructing a simple electrical tester using the wires and battery.\nAction: connect battery cathode to yellow wire terminal 1\nObservation: cathode on battery is now connected to terminal 1 on yellow wire\nThought: I'll choose to connect the blue wire to the red light bulb in order to complete a circuit that may help test the electrical properties of the unknown substance J.\nAction: connect blue wire terminal 2 to cathode in red light bulb\nObservation: terminal 2 on blue wire is now connected to cathode on red light bulb\nThought: Now, I'll connect the green wire to complete the circuit using the red light bulb, a step in testing the electrical conductivity of unknown substance J.\nAction: connect green wire terminal 2 to anode in red light bulb\nObservation: terminal 2 on green wire is now connected to anode on red light bulb\nThought: I continue connecting unknown substance J to the yellow wire to test its conductivity by seeing if it completes a circuit when power is applied.\nAction: connect unknown substance J terminal 1 to yellow wire terminal 2\nObservation: terminal 1 on unknown substance J is now connected to terminal 2 on yellow wire\nThought: I'll now connect the other terminal of the unknown substance J to the green wire, which is connected to the anode of the green light bulb, to complete the circuit through the substance.\nAction: connect unknown substance J terminal 2 to green wire terminal 1\nObservation: terminal 2 on unknown substance J is now connected to terminal 1 on green wire\nThought: I choose to wait because I need to observe the effects of my previous action connecting the wires before proceeding further.\nAction: wait1\nObservation: You decide to wait for 1 iterations.\nThought: I am taking a moment to observe and understand the environment and the items available before proceeding with the task.\nAction: wait1\nObservation: You decide to wait for 1 iterations.\nThought: I chose to look around to get a detailed view of the environment and identify if there are any tools that can help me test the electrical conductivity of unknown substance J.\nAction: look around\nObservation: This room is called the workshop. In it, you see: \n\ta table. On the table is: a yellow light bulb, which is off, a switch, which is off, a red light bulb, which is off, a green wire, a yellow wire, a battery, a blue wire, a blue light bulb, which is off.\n\ta green box (containing nothing)\n\ta ultra low temperature freezer. The ultra low temperature freezer door is closed. \n\ta substance called air\n\ta red box (containing nothing)\n\tunknown substance J\n\tthe agent\nYou also see:\n\tA door to the hallway (that is open)\nThought: I chose to move unknown substance J to the green box because I assumed it to be non-conductive due to lack of evidence of its conductivity.\nAction: move unknown substance J to green box", \
'4-1': "", \
'4-2': "", \
'4-3': "", \
'4-4': "", \
'5-1': "", \
'5-2': "", \
'6-1': "Thought: Since the task mentions that a recipe and some ingredients might be found near the kitchen. I need to go to the kitchen. First, I need to identify my surroundings and locate potential ingredients and equipment for creating sodium acetate.\nAction: look around\nThis room is called the art studio. In it, you see: \n\ta large cupboard. The large cupboard door is closed. \n\ta wood cup (containing red paint)\n\tthe agent\n\ta substance called air\n\ta table. On the table is: a bowl (containing nothing).\n\ta wood cup (containing blue paint)\n\ta wood cup (containing yellow paint)\nYou also see:\n\tA door to the hallway (that is open)\nThought: The art studio doesn't have the necessary ingredients for soapy water. I should go to the kitchen where cleaning supplies are likely to be found.\nAction: teleport to kitchenObservation: You teleport to the kitchen.\nThought: Now that I'm in the kitchen, I should search for the ingredients and tools needed to make soapy water as per the task description.\nAction: look around\nObservation: This room is called the kitchen. In it, you see: \n\tthe agent\n\ta substance called air\n\ta chair. On the chair is: nothing.\n\ta counter. On the counter is: a bowl (containing a red apple, a banana, an orange, a potato), a drawer.\n\ta cupboard. The cupboard door is closed.\n\ta freezer. The freezer door is open. In the freezer is: nothing.\n\ta fridge. The fridge door is open. In the fridge is: a wood cup (containing orange juice).\n\ta glass jar (containing a substance called sodium chloride)\n\ta lighter\n\ta oven, which is turned off. The oven door is open. In the oven is: nothing.\n\ta painting\n\tA recipe titled instructions to make soapy water\n\ta sink, which is turned off. In the sink is: nothing.\n\ta substance called soap\n\ta stopwatch, which is deactivated. \n\ta stove, which is turned off. On the stove is: nothing.\n\ta table. On the table is: a glass cup (containing nothing).\n\ta thermometer, currently reading a temperature of 10 degrees celsius\nYou also see:\n\tA door to the bathroom (that is open)\n\tA door to the hallway (that is open)\n\tA door to the outside (that is open)\nThought: I need open the cupboard to look for ingredients that might help me create soapy water.\nAction: open cupboard\nObservation: The cupboard is now open. In the cupboard is: a ceramic cup (containing nothing), a drawer, a metal pot (containing nothing), a tin cup (containing nothing).\nThought: I chose to pick up the ceramic cup because it could be useful as a container to mix ingredients in to create the soapy water.\nAction: pick up ceramic cup\nObservation: You move the ceramic cup to the inventory.\nThought: I chose to read the recipe to understand the steps and ingredients needed to create soapy water, ensuring I follow the right process.\nAction: read recipe in inventory\nObservation: The recipe reads:\nTo make soapy water, you need to mix soap, water.\nThought: I chose to pick up the soap as it is a necessary ingredient for making soapy water according to the task description.\nAction: pick up soap\nObservation: You move the soap to the inventory.\nThought: I need put the soap into the ceramic cup to start following the recipe steps for making soapy water.\nAction: move soap to ceramic cup\nObservation: You move the soap to the ceramic cup.\nThought: I'll movie the ceramic cup with soap to the sink so that I can add water to the cup to make soapy water.\nAction: move ceramic cup to sink\nObservation: You move the ceramic cup to the sink.\nThought: By activating the sink, I plan to fill the ceramic cup, which contains soap, with water to create soapy water as per the instructions.\nAction: activate sink\nObservation: The sink is now activated.\nThought: I'll deactivate the sink to prevent water waste while I prepare to mix the necessary ingredients to create soapy water.\nAction: deactivate sink\nObservation: The sink is now deactivated.\nThought: I chose to pick up the ceramic cup because it contains the ingredients water and soap, which are needed to make soapy water according to the recipe.\nAction: pick up ceramic cup\nObservation: You move the ceramic cup to the inventory.\nThought: I decided to examine the ceramic cup because I need to check its contents and verify whether the mixture of water and soap has correctly formed soapy water.\nAction: examine ceramic cup\nObservation: a ceramic cup (containing a substance called water, a substance called soap)\nThought: I chose to mix the ceramic cup because it contains water and soap, the necessary ingredients for creating soapy water, according to the recipe.\nAction: mix ceramic cup\nObservation: Soap and water mix to produce soapy water.\nThought: I chose to focus on the soapy water to confirm the successful creation and completion of my given task.\nAction: focus on soapy water.", \
'6-2': "Thought: To create green-blue paint, I need to first find the primary colors that can be mixed to create it. I should look around to identify my resources and the environment I'm in.\nAction: look around\nThis room is called the hallway. In it, you see: \n\tthe agent\n\ta substance called air\n\ta picture\nYou also see:\n\tA door to the green house (that is open)\n\tA door to the living room (that is open)\n\tA door to the art studio (that is open)\n\tA door to the kitchen (that is open)\n\tA door to the bedroom (that is open)\n\tA door to the workshop (that is open)\nThought: I can go to the art studio because it likely has materials and tools necessary for creating paint.\nAction: teleport to art studio\nObservation: You teleport to the art studio.\nThought: I can look around to identify the resources and tools available in the art studio that could help in making green-blue paint.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta wood cup (containing red paint)\n\tthe agent\n\ta wood cup (containing blue paint)\n\ta table. On the table is: a jug (containing nothing).\n\ta paint cupboard. The paint cupboard door is closed. \n\ta wood cup (containing yellow paint)\n\ta substance called air\n\ta large cupboard. The large cupboard door is closed. \nYou also see:\n\tA door to the hallway (that is open)\nThought: I chose to pour the blue paint into the jug as I need to mix blue with another color to create green-blue paint.\nAction: pour cup containing blue paint in art studio in jug\nObservation: You pour the contents of the wood cup into the jug.\nThought: I should mix yellow and blue paint to create green, which is part of the process to eventually produce a green-blue paint.\nAction: pour cup containing yellow paint in art studio in jug\nObservation: You pour the contents of the wood cup into the jug.\nThought: I chose to mix the jug to combine the yellow paint with blue paint, creating green paint which is a key step towards achieving the green-blue paint required.\nAction: mix jug\nObservation: Blue and yellow paint mix to produce green paint.\nThought: I can look around to gather more information about the environment and see if there are any additional resources or tools that could be helpful for completing my task.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta wood cup (containing red paint)\n\tthe agent\n\ta wood cup (containing nothing)\n\ta table. On the table is: a jug (containing green paint).\n\ta paint cupboard. The paint cupboard door is closed. \n\ta wood cup (containing nothing)\n\ta substance called air\n\ta large cupboard. The large cupboard door is closed. \nYou also see:\n\tA door to the hallway (that is open)\nThought: I chose to focus on the green paint because it is a component of my task to create green-blue paint, and focusing could help me understand how far along the paint mixing process I am or decide next steps.\nAction: focus on green paint\nObservation: You focus on the green paint.\nThought: I decide to wait because I may need more information or changes in the environment before proceeding with my task.\nAction: wait1\nObservation: You decide to wait for 1 iterations.\nThought: I chose to open the large cupboard to search for blue paint that is needed to create the desired green-blue paint.\nAction: open large cupboard\nObservation: The large cupboard is now open. In the large cupboard is: a drawer, a wood cup (containing red paint), a wood cup (containing blue paint), a wood cup (containing yellow paint).\nThought: I chose to pour the blue paint into the jug containing green paint to mix them, aiming to create a green-blue paint as per the task description.\nAction: pour cup containing blue paint in jug\nObservation: You pour the contents of the wood cup into the jug.\nThought: I chose to mix the jug to combine the blue paint with the green paint in it, aiming to create a green-blue color as required for completing the task.\nAction: mix jug\nObservation: Blue and green paint mix to produce blue green paint.\nThought: The green-blue paint is now complete, and I need to focus on this final product as the task is now finished.\nAction: focus on green-blue paint.", \
'6-3': "Thought: I need to find materials to create green paint, which are likely to be found in an art studio. Now, I'll look around the current environment to identify potential resources.\nAction: look around\nThis room is called the hallway. In it, you see: \n\ta substance called air\n\tthe agent\n\ta picture\nYou also see:\n\tA door to the green house (that is open)\n\tA door to the living room (that is open)\n\tA door to the art studio (that is open)\n\tA door to the kitchen (that is open)\n\tA door to the bedroom (that is open)\n\tA door to the workshop (that is open)\nThought: The hallway doesn't have the materials for creating green paint. I chose to go to the art studio because it is likely to have materials needed for creating green paint.\nAction: teleport to art studio\nObservation: You teleport to the art studio.\nThought: I chose to look around to identify materials and tools available in the art studio that could help me create green paint.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta table. On the table is: a jug (containing nothing).\n\ta wood cup (containing blue paint)\n\ta large cupboard. The large cupboard door is closed. \n\ta wood cup (containing yellow paint)\n\tthe agent\n\ta wood cup (containing red paint)\n\ta substance called air\n\ta paint cupboard. The paint cupboard door is closed. \nYou also see:\n\tA door to the hallway (that is open)\nThought: I chose to pour the blue paint into the jug because I need to mix it with another color to create green paint.\nAction: pour cup containing blue paint in art studio in jug\nObservation: You pour the contents of the wood cup into the jug.\nThought: I chose to pour yellow paint into the jug containing blue paint because mixing yellow and blue will create green paint.\nAction: pour cup containing yellow paint in art studio in jug\nObservation: You pour the contents of the wood cup into the jug.\nThought: I chose to mix the jug to combine the blue and yellow paints to create green paint.\nAction: mix jug\nObservation: Blue and yellow paint mix to produce green paint.\nThought: I chose to look around to check the generation of the green paint.\nAction: look around\nObservation: This room is called the art studio. In it, you see: \n\ta table. On the table is: a jug (containing green paint).\n\ta wood cup (containing nothing)\n\ta large cupboard. The large cupboard door is closed. \n\ta wood cup (containing nothing)\n\tthe agent\n\ta wood cup (containing red paint)\n\ta substance called air\n\ta paint cupboard. The paint cupboard door is closed. \nYou also see:\n\tA door to the hallway (that is open)\nThought: I chose to focus on the green paint because the observation revealed that there is already green paint in the jug on the table, which meets the task requirement.\nAction: focus on green paint", \
'7-1': "Thought: The task is to find the animal with the longest lifespan, and I need to go to the outside location where the animals are to identify which one has the longest lifespan. So, I should start by looking around the current location to identify a way to get outside.\nAction: look around\nObservation: This room is called the hallway. In it, you see: \n\ta substance called air\n\ta picture\n\tthe agent\nYou also see:\n\tA door to the green house (that is open)\n\tA door to the living room (that is open)\n\tA door to the art studio (that is open)\n\tA door to the kitchen (that is open)\n\tA door to the bedroom (that is open)\n\tA door to the workshop (that is open)\nThought: Going outside is essential to find the animal with the longest lifespan, as mentioned in the task description that the animals are located there.\nAction: teleport to outside\nObservation: You teleport to the outside.\nThought: Now that I am outside, I should look around to identify all the animals present here so I can determine which one has the longest lifespan.\nAction: look around\nObservation: This outside location is called the outside. Here you see: \n\tthe agent\n\ta substance called air\n\ta baby ant\n\tan axe\n\ta baby brown bear\n\ta fire pit (containing nothing)\n\ta fountain (containing a substance called water)\n\tthe ground\n\ta baby rabbit\n\ta substance called wood\nYou also see:\n\tA door to the foundry (that is open)\n\tA door to the greenhouse (that is open)\n\tA door to the kitchen (that is open)\nThought: I will focus on the baby brown bear to determine if it might be the animal with the longest lifespan in this area.\nAction: focus on baby brown bear\nObservation: You focus on the baby brown bear.", \
'7-2': "", \
'7-3': "", \
'8-1': "", \
'8-2': "Thought: To observe the environment and identify the presence of turtle life stages, I need to first assess my surroundings.\nAction: look around\nObservation: This room is called the hallway. In it, you see: \n\ta substance called air\n\tthe agent\n\ta picture\nYou also see:\n\tA door to the green house (that is open)\n\tA door to the living room (that is open)\n\tA door to the art studio (that is open)\n\tA door to the kitchen (that is open)\n\tA door to the bedroom (that is open)\n\tA door to the workshop (that is open)\nThought: I want to find information about turtle life stages, possibly outside or in a more relevant room.\nAction: teleport to outside\nObservation: You teleport to the outside.\nThought: I chose to look around to gather information about my new surroundings and potentially find clues to help complete the task related to turtle life stages.\nAction: look around\nObservation: This outside location is called the outside. Here you see: \n\ta butterfly egg\n\ta substance called air\n\ta parrot egg\n\ta fountain (containing a substance called water)\n\ta giant tortoise egg\n\ta fire pit (containing nothing)\n\tan axe\n\tthe ground\n\ta turtle egg\n\ta baby wolf\n\tthe agent\n\ta baby beaver\n\ta substance called wood\nYou also see:\n\tA door to the foundry (that is open)\n\tA door to the green house (that is open)\n\tA door to the kitchen (that is open)\nThought: I think the turtle egg aligns with the first life stage of a turtle, which is the beginning of my task to explore all life stages of the turtle. Thus, I chose to focus on it.\nAction: focus on egg turtle in outside\nObservation: You focus on the turtle egg.\nThought: I need wait to observe any changes in the environment that might help me focus on the turtle life stages.\nAction: wait\nObservation: You decide to wait for 10 iterations.\nThought: Some time has passed. I should looking around to gather more information about the other life stages of the turtle present here.\nAction: look around\nObservation: This outside location is called the outside. Here you see: \n\ta caterpillar\n\ta substance called air\n\ta hatchling parrot\n\ta fountain (containing a substance called water)\n\ta hatchling giant tortoise\n\ta fire pit (containing nothing)\n\tan axe\n\tthe ground\n\ta hatchling turtle\n\ta juvenile wolf\n\tthe agent\n\ta juvenile beaver\n\ta substance called wood\nYou also see:\n\tA door to the foundry (that is open)\n\tA door to the green house (that is open)\n\tA door to the kitchen (that is open)\nThought: The turtle egg has hatched into a hatchling. I chose to focus on the hatchling stage of a turtle.\nAction: focus on hatchling turtle in outside\nObservation: You focus on the hatchling turtle.\nThought: I chose to wait again to observe any changes in the environment that might aid in identifying the life stages of the turtle.\nAction: wait\nObservation: You decide to wait for 10 iterations.\nThought: After waiting, I should look around to see if the hatchling turtle has grown into the next life stage.\nAction: look around\nObservation: This outside location is called the outside. Here you see: \n\ta caterpillar\n\ta substance called air\n\ta hatchling parrot\n\ta fountain (containing a substance called water)\n\ta hatchling giant tortoise\n\ta fire pit (containing nothing)\n\tan axe\n\tthe ground\n\ta hatchling turtle\n\ta juvenile wolf\n\tthe agent\n\ta juvenile beaver\n\ta substance called wood\nYou also see:\n\tA door to the foundry (that is open)\n\tA door to the green house (that is open)\n\tA door to the kitchen (that is open)\nThought: The hatchling turtle has not yet grown. I chose to wait again to observe if any changes occur that might help understand the life stages of the turtle better.\nAction: wait\nObservation: You decide to wait for 10 iterations.\nThought: More time has elapsed, which may be sufficient for the hatchling to grow. I should look around to check for progress.\nAction: look around\nObservation: This outside location is called the outside. Here you see: \n\ta butterfly pupa\n\ta substance called air\n\ta juvenile parrot\n\ta fountain (containing a substance called water)\n\ta juvenile giant tortoise\n\ta fire pit (containing nothing)\n\tan axe\n\tthe ground\n\ta juvenile turtle\n\ta adult wolf\n\tthe agent\n\ta adult beaver\n\ta substance called wood\nYou also see:\n\tA door to the foundry (that is open)\n\tA door to the green house (that is open)\n\tA door to the kitchen (that is open)\nThought: The hatchling has grown into a juvenile turtle. I can focus on the juvenile turtle to observe.\nAction: focus on juvenile turtle in outside\nObservation: You focus on the juvenile turtle.\nThought: Waiting again allows for further observation of the environment and its changes, particularly noting any developments in the juvenile turtle's behavior or surroundings.\nAction: wait\nObservation: You decide to wait for 10 iterations.\nThought: I should look around again to see if the juvenile turtle has matured into an adult after the waiting period.\nAction: look around\nObservation: This outside location is called the outside. Here you see: \n\ta butterfly pupa\n\ta substance called air\n\ta juvenile parrot\n\ta fountain (containing a substance called water)\n\ta juvenile giant tortoise\n\ta fire pit (containing nothing)\n\tan axe\n\tthe ground\n\ta juvenile turtle\n\ta adult wolf\n\tthe agent\n\ta adult beaver\n\ta substance called wood\nYou also see:\n\tA door to the foundry (that is open)\n\tA door to the green house (that is open)\n\tA door to the kitchen (that is open)\nThought: I need wait again because I have not yet identified any changes occur with the juvenile turtle that might indicate moving to a different life stage.\nAction: wait\nObservation: You decide to wait for 10 iterations.\nThought: Sufficient time may have passed for the juvenile turtle to become an adult. I should look around to confirm this.\nAction: look around\nObservation: This outside location is called the outside. Here you see: \n\ta adult butterfly\n\ta substance called air\n\ta adult parrot\n\ta fountain (containing a substance called water)\n\ta adult giant tortoise\n\ta fire pit (containing nothing)\n\tan axe\n\tthe ground\n\ta adult turtle\n\ta adult wolf\n\tthe agent\n\ta adult beaver\n\ta substance called wood\nYou also see:\n\tA door to the foundry (that is open)\n\tA door to the green house (that is open)\n\tA door to the kitchen (that is open)\nThought: Now I can focus on the adult turtle in the outside area because my task is to observe the life stages of turtles, and an adult turtle is a crucial stage in their lifecycle.\nAction: focus on adult turtle in outside."
}
example_plan = {"1-1": "Step 1. go_to_the_place_suitable_for_boiling_orange_juice(\"kitchen\");\nStep 2. search_for_tools(\"stove\");\nStep 3. pick_up(\"metal pot\");\nStep 4. look_for(\"orange juice\");\nStep 5. focus_on(\"orange juice\");\nStep 6. pour_substance_into_container(\"orange juice\", \"metal pot\");\nStep 7. pick_up(\"metal pot\");\nStep 8. move_metal_pot_to_stove();\nStep 9. activate(\"stove\");\nStep 10. if stove_is_broken, pick_up(\"metal pot\") and then heat_source = find_alternative_for_heating(); move_metal_pot_to_heat_source(); activate(heat_source);\nStep 11. examine(\"orange juice\");\nStep 12. repeat Step 11 until orange juice changes its state of matter.",\
"1-2": "", \
"1-3": "Step 1. go_to_the_place_suitable_for_freezing_chocolate(\"kitchen\");\nStep 2. look_around_to_check();\nStep 3. search_for_tools(\"pot\");\nStep 4. pick_up(\"metal pot\");\nStep 5. look_for(\"chocolate\");\nStep 6. focus_on(\"chocolate\");\nStep 7. move_chocolate_to_metal_pot();\nStep 8. pick_up(\"metal pot\");\nStep 9. move_metal_pot_to_stove();\nStep 10. activate_stove_to_heat_chocolate();\nStep 11. examine(\"chocolate\");\nStep 12. repeat Step 11 until reaching liquid chocolate;\nStep 13. pick_up(\"metal pot\");\nStep 14. open(\"freezer\");\nStep 15. move_metal_pot_to_freezer();\nStep 16. examine(\"chocolate\");\nStep 17. wait_to_observe();\nStep 18. repeat step 16,17 until reaching freezing chocolate.", \
'1-4': "Step 1. go_to_the_place_suitable_for_task(\"kitchen\");\nStep 2. look_around_to_check();\nStep 3. search_for_tools(\"pot\");\nStep 4. pick_up(\"metal pot\");\nStep 5. look_for(\"chocolate\");\nStep 6. focus_on(\"chocolate\");\nStep 7. move_chocolate_to_metal_pot();\nStep 8. pick_up(\"metal pot\");\nStep 9. move_metal_pot_to_stove();\nStep 10. activate(\"stove\");\nStep 11. examine_substance_in(\"metal pot\");\nStep 12. repeat step 11 or wait_to_observe_state_change();", \
'2-1': "Step 1. go_to_the_place_with_thermometer(\"kitchen\");\nStep 2. search_for(\"thermometer\");\nStep 3. pick_up(\"thermometer\");\nStep 4. focus_on_to_check(\"thermometer\");\nStep 5. go_to_the_place_with_unknown_substance(\"living room\");\nStep 6. search_for(\"unknown substance B\");\nStep 7. pick_up(\"unknown substance B\");\nStep 8. focus_on(\"unknown substance B\");\nStep 9. go_to_the_place_with_boxes(\"bathroom\");\nStep 10. search_for(\"red box\", \"green box\");\nStep 11. temperature = measure_temperature(\"unknown substance B\", \"thermometer\");\nStep 12. if temperature > 100.0 degrees, focus_on(\"red box\");\n    else: focus_on(\"green box\")", \
'2-2': "Step 1. go_to_the_place_with_thermometer_and_chocolate(\"kitchen\");\nStep 2. search_for_tools(\"thermometer\", \"stove\", \"blue box\", \"orange box\");\nStep 3. pick_up(\"thermometer\");\nStep 4. focus_on_to_check(\"thermometer\");\nStep 5. search_for(\"chocolate\");\nStep 6. focus_on(\"chocolate\");\nStep 7. pick_up(\"metal pot\");\nStep 8. move_chocolate_to_metal_pot();\nStep 9. move_metal_pot_to_stove();\nStep 10. activate_stove_to_heat_pot();\nStep 11. examine(\"chocolate\");\nStep 12. temperature = measure_temperature(\"chocolate\", \"thermometer\")\nStep 13. repeat step 11,12 until getting enough information about the melting point;\nStep 14. if melting_point(\"chocolate\") > -10.0 degrees, focus_on(\"blue box\");\n    else: focus_on(\"orange box\").", \
'3-1': "", \
'3-2': "Step 1. look_around_to_check_current_place();\nStep 2. go_to(\"workshop\");\nStep 3. look_for_tools(\"red light bulb\");\nStep 4. focus_on(\"red light bulb\");\nStep 5. look_for_tools(\"nonrenewable power source\", \"wires\");\nStep 6. connect_power_source_anode_to_one_wire_terminal();\nStep 7. connect_power_source_cathode_to_another_wire_terminal();\nStep 8. connect_wires_to_light_bulb(\"anode to cathode and vice versa\");\nStep 9. activate(power_source);\nStep 10. wait_to_observe().", \
'3-3': "", \
'3-4': "Step 1. look_around_to_check_current_place();\nStep 2. go_to(\"workshop\");\nStep 3. pick_up(\"unknown substance\");\nStep 4. focus_on(\"unknown substance\");\nStep 5. drop_on_the_table_for_testing_conductivity(\"unknown substance\");\nStep 6. look_for_tools(\"wires\", \"battery\", \"light bulb\");\nStep 7. connect_battery_anode_to_one_wire_terminal();\nStep 8. connect_battery_cathode_to_another_wire_terminal();\nStep 9. connect_wires_to_light_bulb(\"anode to cathode and vice versa\");\nStep 10. connect_unknown_substance_to_the_test_circuit();\nStep 11. wait();\nStep 12. current_observation = look_around();\nStep 13. if \"light bulb turns on\" in current_observation: move_unknown_substance_J_to_red_box();\n    else:  move_unknown_substance_J_to_green_box().", \
'4-1': "", \
'4-2': "", \
'4-3': "", \
'4-4': "", \
'5-1': "", \
'5-2': "", \
'6-1': "Step 1. find_way_to_the_place_with_recipe_and_ingredients(\"kitchen\");\nStep 2. go_to(\"kitchen\");\nStep 3. mixing_container = find_mixing_container();\nStep 4. pick_up(mixing_container);\nStep 5. recipe = look_around_to_gather_information();\nStep 6. pick_up(recipe);\nStep 7. necessary_ingredients = read(recipe);\nStep 8. necessary_ingredients, water_source = look_around_to_gather_information();\nStep 9. if necessary_ingredients in containers, open(containers);\nStep 10. pick_up(necessary_ingredients);\nStep 11. move(necessary_ingredients, mixing_container);\nStep 12. move(mixing_container, water_source);\nStep 13. activate(water_source);\nStep 14. deactivate(water_source);\nStep 15. pick_up(mixing_container);\nStep 16. examine_substance(mixing_container);\nStep 17. mix(ingredients_in_mixing_container);\nStep 18. focus_on(\"soapy water\").", \
'6-2': "Step 1. place = explore_places_with_necessary_ingredients_or_tools(\"blue paint\",\"green paint\", \"a mixing container\");\nStep 2. go_to(place);\nStep 3. look_around_to_gather_information();\nStep 4. repeat step 1, 2, 3 until necessary ingredients and tools found;\nStep 5. pour_blue_paint_into_mixing_container();\nStep 6. pour_yellow_paint_into_the_same_mixing_container();\nStep 7. mix_paints();\nStep 8. look_around_to_observe(\"green paint\");\nStep 9. focus_on(\"green paint\");\nStep 10. wait_to_verify();\nStep 11. search_for_additional_supplies(\"blue paint\");\nStep 12. pour_blue_paint_into_the_same_mixing_container();\nStep 13. mix_paints();\nStep 14. look_around_to_observe(\"green-blue paint\");\nStep 15. focus_on(\"green-blue paint\").", \
'6-3': "Step 1. place = explore_places_with_necessary_ingredients_or_tools(\"blue paint\",\"yellow paint\", \"a mixing container\");\nStep 2. go_to(place);\nStep 3. look_around_to_gather_information();\nStep 4. repeat step 1, 2, 3 until necessary ingredients and tools found;\nStep 5. pour_blue_paint_into_mixing_container();\nStep 6. pour_yellow_paint_into_the_same_mixing_container();\nStep 7. mix_paints();\nStep 8. look_around_to_observe(\"green paint\");\nStep 9. focus_on(\"green paint\").", \
'7-1': "Step 1. look_around_to_check_current_location();\nStep 2. go_to_the_outside();\nStep 3. look_around_to_observe();\nStep 4. focus_on(\"animal with the longest life span\").", \
'7-2': "", \
'7-3': "", \
'8-1': "", \
'8-2': "Step 1. go_to_the_outside();\nStep 2. look_around_to_gather_information(\"turtle life stages\");\nStep 3. focus_on(\"the first life stage of a turtle\");\nStep 4. wait_to_observe();\nStep 5.look_around_to_gather_information(\"turtle life stages\");\nStep 6. repeat step 4 or 5 until find and focus_on(\"the second life stage of a turtle\");\nStep 7. wait_to_observe();\nStep 8. look_around_to_gather_information(\"turtle life stages\");\nStep 9. repeat step 7 or 8 until find and focus_on(\"the third life stage of a turtle\");\nStep 10. wait_to_observe();\nStep 11. look_around_to_gather_information(\"turtle life stages\");\nStep 12. repeat step 10 or 11 until find and focus_on(\"the fourth life stage of a turtle\")."
}
for id in ['1-2']:
    example_task[id]=example_task['1-3']
    example_traj[id]=example_traj['1-3']
    example_plan[id]=example_plan['1-3']
for id in ['3-1']:
    example_task[id]=example_task['3-2']
    example_traj[id]=example_traj['3-2']
    example_plan[id]=example_plan['3-2']
for id in ['3-3']:
    example_task[id]=example_task['3-4']
    example_traj[id]=example_traj['3-4']
    example_plan[id]=example_plan['3-4']
for id in ['4-1','4-2','4-3','4-4','7-2','7-3']:
    example_task[id]=example_task['7-1']
    example_traj[id]=example_traj['7-1']
    example_plan[id]=example_plan['7-1']
for id in ['8-1']:
    example_task[id]=example_task['8-2']
    example_traj[id]=example_traj['8-2']
    example_plan[id]=example_plan['8-2']
# example_plan = ""
task_ids = ['1-1','1-2','1-3','1-4','2-1','2-2','3-1','3-2','3-3','3-4','4-1','4-2','4-3','4-4','5-1','5-2','6-1','6-2','6-3','7-1','7-2','7-3','8-1','8-2']
results = []
with open('./data/pcode_plan_prompts/sciworld_plan_extraction_prompt.json', 'w') as f:
    for id in task_ids:
        icl_example_demo = icl_example.replace("<task>",example_task[id]).replace("<traj>",example_traj[id]).replace("<plan>",example_plan[id])
        print(icl_example_demo)
        result = {"task_id":id,"example_task":example_task[id],"example_traj":example_traj[id], "example_plan":example_plan[id],"icl_example":icl_example_demo}
        results.append(result)
    json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
icl_examples = {}
with open('./data/pcode_plan_prompts/sciworld_plan_extraction_prompt.json', encoding="utf-8") as f:
    data = json.load(f)
    for line in data:
        icl_examples[line['task_id']]=line['icl_example']

output_path = "./data/sciworld_sft_with_plan.json"
data_path = "./data/ReAct_style_data/sciworld_sft.json"
with open(output_path,'w',encoding='utf-8') as f:
    results = []
    with open(data_path, encoding="utf-8") as f:
        data=json.load(f)
        for line in data:
            id = line['id']
            convs = line['conversations']
            sys_prompt = "[Task Description]:\n" + convs[0]['value']
            task = "\n\n[Task]:\n" + convs[2]['value']
            # print(task)
            task_id = getTaskType(task)
            icl_example = icl_examples[task_id]
            trajs = "\n\n[Solution Trajectory]:\n"
            for conv in convs[3:]:
                trajs += conv['value'] + "\n"
            icl_example = icl_example.replace("<sys_prompt>",sys_prompt)
            query = sys_prompt + task + trajs + "\n[Plan]:"
            input_prompt = extract_plan_prompt + icl_example + query
            response = llm._call(input_prompt)
            if response is not None:
                plan = "Thought: First, I devise a plan for solving the task:\n"+response
                line['conversations'][3]['value'] = plan + "\n" + line['conversations'][3]['value'][9:]
                results.append(line)
            else:
                print(f"error meets when id={id}")
    json.dump(results, f, ensure_ascii=False, indent=4)

## **Compared with NL Plan**

### **ALFWorld**

In [ ]:
icl_examples = {}
with open('./data/nl_plan_prompts/alfworld_plan_extraction_prompt.json', encoding="utf-8") as f:
    data = json.load(f)
    for line in data:
        icl_examples[line['task_id']]=line['icl_example']

data_path = "./data/ReAct_style_data/alfworld_sft.json"
output_path = "./data/data_with_nl_plan/alfworld_sft_with_nl_plan.json"
with open(output_path,'w',encoding='utf-8') as f:
    results = []
    with open(data_path, encoding="utf-8") as f:
        data=json.load(f)
        for line in data:
            id = line['id']
            game_file = line['game_file']
            convs = line['conversations']
            sys_prompt = "[Task Description]:\n" + convs[0]['value']
            task = "\n\n[Task]:\n" + convs[2]['value']
            task_id = getTaskType(task)
            icl_example = icl_examples[task_id]
            trajs = "\n\n[Solution Trajectory]:\n"
            for conv in convs[3:]:
                trajs += conv['value'] + "\n"
            icl_example = icl_example.replace("<sys_prompt>",sys_prompt)
            query = sys_prompt + task + trajs + "\n[Plan]:"
            input_prompt = extract_plan_prompt + icl_example + query
            response = llm._call(input_prompt)
            if response is not None:
                plan = "Thought: First, I devise a plan for solving the task:\n"+response
                line['conversations'][3]['value'] = plan + "\n" + line['conversations'][3]['value'][9:]
                results.append(line)
            else:
                print(f"error meets when id={id}")
    json.dump(results, f, ensure_ascii=False, indent=4)

### **WebShop**

In [ ]:
with open('./data/nl_plan_prompts/webshop_plan_extraction_prompt.json', encoding="utf-8") as f:
    data = json.load(f)[0]
    icl_example=data['icl_example']

data_path = "./data/ReAct_style_data/webshop_sft.json"
output_path = "./data/data_with_nl_plan/webshop_sft_with_nl_plan.json"
data_to_upload = []
with open(output_path,'w',encoding='utf-8') as f:
    results = []
    with open(data_path, encoding="utf-8") as f:
        data=json.load(f)
        for line in data:
            id = line['id']
            convs = line['conversations']
            sys_prompt = "[Task Description]:\n" + convs[0]['value']
            task = "\n\n[Task]:\n" + convs[2]['value']
            trajs = "\n\n[Solution Trajectory]:\n"
            for conv in convs[3:]:
                trajs += conv['value'] + "\n"
            icl_example = icl_example.replace("<sys_prompt>",sys_prompt)
            query = sys_prompt + task + trajs + "\n[Plan]:"
            input_prompt = extract_plan_prompt + icl_example + query
            # print(input_prompt)
            data_to_upload.append([id, input_prompt])
            response = llm._call(input_prompt)
            if response is not None:
                plan = "Thought: First, I devise a plan for solving the task:\n"+response
                line['conversations'][3]['value'] = plan + "\n" + line['conversations'][3]['value'][9:]
                results.append(line)
            else:
                print(f"error meets when id={id}")
    json.dump(results, f, ensure_ascii=False, indent=4)

## **Compared with Execode Plan**

In [ ]:
extract_plan_prompt = "Given the [Task Description], [Task] and [Solution Trajectory], you should extract the step-by-step [Plan] for solving the task. Please note that the extracted [Plan] should be global and do not contain the information from \"Observation\" part of [Solution Trajectory]. Your extracted [Plan] must strictly follow the python code format.\n"

### **ALFWorld**

In [ ]:
icl_example = "Here is one example.\n<sys_prompt>\n\n[Task]:\n<task>\n\n[Solution Trajectory]:\n<traj>\n\n[Plan]:\n<plan>\n\nNow is your turn.\n" 
example_task = {"put": "You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a shelf 2, a shelf 1, and a sidetable 1.\nYour task is to: put some alarmclock on desk.", \
"clean": "You are in the middle of a room. Looking quickly around you, you see a cabinet 21, a cabinet 20, a cabinet 19, a cabinet 18, a cabinet 17, a cabinet 16, a cabinet 15, a cabinet 14, a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: clean some apple and put it in diningtable.", \
"cool": "You are in the middle of a room. Looking quickly around you, you see a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: cool some apple and put it in diningtable.", \
"heat": "You are in the middle of a room. Looking quickly around you, you see a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 1, a diningtable 1, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: heat some apple and put it in diningtable.", \
"examine": "You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 17, a drawer 16, a drawer 15, a drawer 14, a drawer 13, a drawer 12, a drawer 11, a drawer 10, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.\nYour task is to: look at alarmclock under the desklamp."
}
example_traj = {"put": "Thought: I need to find an alarmclock and put it on the desk. Alarmclocks usually located on bedside tables or in the drawers. Therefore, I will check the drawers first.\nAction: go to drawer 1\nObservation: The drawer 1 is closed.\nThought: Since the drawer 1 is closed, I need open it.\nAction: open drawer 1\nObservation: You open the drawer 1. The drawer 1 is open. In it, you see a remotecontrol 1.\nThought: Drawer 1 only has a remotecontrol. I need to continue searching the other drawers for the alarmclock.\nAction: go to drawer 2\nObservation: The drawer 2 is closed.\nThought: I need to open the drawer 2.\nAction: open drawer 2\nObservation: You open the drawer 2. The drawer 2 is open. In it, you see a book 1.\n No alarmclock is found in drawer 2. I need continue searching the other drawers for the alarm clock.\nAction: go to drawer 3\nObservation: The drawer 3 is closed.\nThought: I need to open the drawer 3.\nAction: open drawer 3\nObservation: You open the drawer 3. The drawer 3 is open. In it, you see a pencil 1.\nThought: I can't find any alarmclock in the drawers. I'll check the shelves next.\nAction: go to shelf 1\nObservation: On the shelf 1, you see nothing.\nThought: There is no alarmclock in shelf 1. I need to continue searching the other shelves for the alarmclock.\nAction: go to shelf 2\nObservation: On the shelf 2, you see nothing.\nThought: There's no alarmclock on both shelves. Alarmclock might be on the sidetable.\nAction: go to sidetable 1\nObservation: On the sidetable 1, you see a alarmclock 2, a alarmclock 1, a creditcard 1, a desklamp 1, a keychain 1, and a pen 1.\nThought: I have found an alarmclock on sidetable 1. I should take it so I can put it on desk.\nAction: take alarmclock 1 from sidetable 1\nObservation: You pick up the alarmclock 1 from the sidetable 1.\nThought: With alarmclock 1 in hand, I need to go to the desk to place it on.\nAction: go to desk 1\nObservation: On the desk 1, you see a book 2, a bowl 1, a cd 3, a cd 2, a cd 1, a cellphone 1, a keychain 2, and a pencil 2.\nThought: Now I need to place the alarmclock 1 on the desk 1 to complete the task.\nAction: put alarmclock 1 in/on desk 1.", \
"clean": "Thought: I need to find an apple first. It is likely to be stored in one of the cabinets or on the countertop. I will start by checking the countertop 1.\nAction: go to countertop 1\nObservation: On the countertop 1, you see a apple 2, a bowl 3, a bread 2, a butterknife 1, a cup 1, a fork 1, a knife 3, a knife 1, a saltshaker 1, a spatula 2, a spatula 1, a spoon 1, and a tomato 1.\nThought: I see a apple 2 on the countertop 1.  should pick up it to clean.\nAction: take apple 2 from countertop 1\nObservation: You pick up the apple 2 from the countertop 1.\nThought: Now I have the apple, I need to clean it. It is likely to be stored in one of the cabinets or on the countertop. I will start by checking the sinkbasin 1.\nAction: go to sinkbasin 1\nObservation: On the sinkbasin 1, you see a potato 1.\nThought: I'm at the sinkbasin. I'll use the sink to wash the apple.\nAction: clean apple 2 with sinkbasin 1\nObservation: You clean the apple 2 using the sinkbasin 1.\nThought: Now the apple is clean, I need to put it on the diningtable 1.\nAction: go to diningtable 1\nObservation: On the diningtable 1, you see a bowl 1, a bread 1, a cup 3, a dishsponge 1, a knife 2, a lettuce 1, a peppershaker 2, a peppershaker 1, and a spatula 3.\nThought: I've arrived at the diningtable, and I'll place the clean apple here as required.\nAction: put apple 2 in/on diningtable 1.", \
"cool": "Thought: I need to find an apple first. It is likely to be stored in one of the cabinets or on the countertop. I will start by checking the countertop 1.\nAction: go to countertop 1\nObservation: On the countertop 1, you see a apple 2, a butterknife 1, a mug 1, a pan 1, a plate 2, a plate 1, a potato 2, a potato 1, a spatula 1, and a winebottle 2.\nThought: I have found the apple on countertop 1. I should pick it up to proceed with the task.\nAction: take apple 2 from countertop 1\nObservation: You pick up the apple 2 from the countertop 1.\nThought: Now I have the apple, I need to cool it down. The fridge is likely to be the best place to do this.\nAction: go to fridge 1\nObservation: The fridge 1 is closed.\nThought: The fridge is closed. I need to open it to place the apple inside for cooling.\nAction: open fridge 1\nObservation: You open the fridge 1. The fridge 1 is open. In it, you see an bread 1, an egg 1, a lettuce 1, and a pan 2.\nThought: It's time to place the apple in the fridge 1 to cool it.\nAction: cool apple 2 with fridge 1\nObservation: You cool the apple 2 using the fridge 1.\nThought: Now the apple is cool, I need to put it on the dining table.\nAction: go to diningtable 1\nObservation: On the diningtable 1, you see a apple 3, a bowl 2, a bread 1, a butterknife 3, a butterknife 2, a fork 3, a lettuce 2, a lettuce 1, a peppershaker 2, a potato 3, a saltshaker 2, a soapbottle 3, a soapbottle 2, a spatula 2, and a winebottle 1.\nThought: I am at the dining table. I should put the apple down here to complete the task.\nAction: put apple 2 in/on diningtable 1.", \
"heat": "Thought: I need to heat an apple and put it on the dining table. I'll first look for an apple. The best places to find one would be a cabinet or the fridge.\nAction: go to cabinet 1\nObservation: The cabinet 1 is closed.\nThought: I need to open the cabinet 1 to search for apples.\nAction: open cabinet 1\nObservation: You open the cabinet 1. The cabinet 1 is open. In it, you see nothing.\nThought: The apple isn't in cabinet 1. Now I'll check the other common location: the fridge.\nAction: go to fridge 1\nObservation: The fridge 1 is closed.\nThought: I need to open the fridge 1 to search for apples.\nAction: open fridge 1\nObservation: You open the fridge 1. The fridge 1 is open. In it, you see a apple 3, a apple 2, a bread 3, a cup 2, and a tomato 1.\nThought: There are several items in the fridge 1, including apples. I need to pick up an apple to proceed with the task.\nAction: take apple 2 from fridge 1\nObservation: You pick up the apple 2 from the fridge 1.\nThought: Now that I have the apple, I need to find a way to heat it. The microwave is a suitable appliance for heating food quickly.\nAction: go to microwave 1\nObservation: The microwave 1 is closed.\nThought:Before I can heat the apple, I need to open the microwave since it's closed.\nAction: open microwave 1\ nObservation: You open the microwave 1. The microwave 1 is open.\nThought: With the microwave open, I can now place the apple inside and heat it.\nAction: heat apple 2 with microwave 1\nObservation: You heat the apple 2 using the microwave 1.\nThought: The apple is heated, and the next step in the task is to put it on the diningtable. I need to locate one diningtable.\nAction: go to diningtable 1\nObservation: On the diningtable 1, you see a apple 1, a bread 1, a butterknife 1, a dishsponge 1, a fork 2, a fork 1, a knife 1, a lettuce 1, a pan 1, a pen 3, a pen 2, a pen 1, a pencil 1, a plate 1, a potato 1, a spatula 1, a spoon 1, and a winebottle 1.\nThought: I've arrived at the diningtable 1. I should now complete the task by putting the heated apple on the diningtable 1.\nAction: put apple 2 in/on diningtable 1.", \
"examine":"Thought: To complete the task, I need to find the alarm clock first. I think the alarm clock might be on the desk or in the drawers. I should start with the desk.\nAction: go to desk 1 \nObservation: On the desk 1, you see a alarmclock 1, a bowl 1, a cd 2, a cd 1, a mug 2, a mug 1, and a pencil 1.\nThought: I have found an alarm clock on desk 1, so I should pick up the alarm clock.\nAction: take alarmclock 1 from desk 1 \nObservation: You pick up the alarmclock 1 from the desk 1 \nThought: Now I have the alarm clock. I need to find a lamp to view it under. I think a desklamp would be perfect for this task. I should go to the dresser next.\nAction: go to dresser 1 \nObservation: On the dresser 1, you see a alarmclock 2, a desklamp 1, a keychain 3, a keychain 1, and a watch 1.\nThought: I see the desklamp on dresser 1. I will use it to look at the alarmclock. Action: use desklamp 1"
}
example_plan = {"put": "for loc in [\"sidetable 1\", \"drawer 1\", \"drawer 2\", \"drawer 3\", \"shelf 1\", \"shelf 2\", \"desk 1\", \"laundryhamper 1\", \"garbagecan 1\"]:\n    res = goto(loc)\n    if \"closed\" in res:\n        res = open_receptacle(loc)\n    if \"alarmclock\" in res:\n        break\ntake_from(res, loc)\ngoto(\"desk 1\")\nput(res, \"desk 1\")", \
"clean": "for loc in [\"fridge 1\", \"countertop 1\", \"countertop 2\", *[f\"cabinet {i}\" for i in range(1, 22)], *[f\"drawer {i}\" for i in range(1, 6)]]:\n    res = goto(loc)\n    if \"closed\" in res:\n        res = open_receptacle(loc)\n    if \"apple\" in res:\n        break\ntake_from(res, loc)\ngoto(\"sinkbasin 1\")\nclean(res, \"sinkbasin 1\")\ngoto(\"diningtable 1\")\nput(res, \"diningtable 1\")", \
"cool": "for loc in [\"fridge 1\", \"countertop 1\", \"countertop 2\", *[f\"cabinet {i}\" for i in range(1, 12)], *[f\"drawer {i}\" for i in range(1, 4)]]:\n    res = goto(loc)\n    if \"closed\" in res:\n        res = open_receptacle(loc)\n    if \"apple\" in res:\n        break\ntake_from(res, loc)\ngoto(\"fridge 1\")\ncool(res, \"fridge 1\")\ngoto(\"diningtable 1\")\nput(res, \"diningtable 1\")", \
"heat": "for location in [\"fridge 1\", \"drawer 1\", \"drawer 2\", \"cabinet 1\", \"cabinet 2\", \"cabinet 3\", \"cabinet 4\", \"cabinet 5\", \"cabinet 6\", \"cabinet 7\", \"cabinet 8\", \"cabinet 9\", \"cabinet 10\"]:\n  res = goto(location)\n  if \"apple\" in res:\n    apple_location = location\n    break\ntake_from(res, apple_location)\ngoto(\"microwave 1\")\nheat(res, \"microwave 1\")\ngoto(\"diningtable 1\")\nput(res, \"diningtable 1\")", \
"examine": "for loc_to_check in [\"drawer 1\", \"drawer 2\", \"drawer 3\", \"drawer 4\", \"drawer 5\", \"drawer 6\", \"drawer 7\", \"drawer 8\", \"drawer 9\", \"drawer 10\", \"drawer 11\", \"drawer 12\", \"drawer 13\", \"drawer 14\", \"drawer 15\", \"drawer 16\", \"drawer 17\", \"shelf 1\", \"shelf 2\", \"shelf 3\", \"shelf 4\", \"shelf 5\", \"shelf 6\", \"desk 1\"]:\n    res = goto(loc_to_check)\n    if \"closed\" in res:\n        res = open_receptacle(loc_to_check)\n    if \"alarmclock\" in res:\n        break\ntake_from(res, loc_to_check)\nfor loc_to_check in [\"desk 1\", \"dresser 1\", \"shelf 1\", \"shelf 2\", \"shelf 3\", \"shelf 4\", \"shelf 5\", \"shelf 6\"]:\n    res = goto(loc_to_check)\n    if \"closed\" in res:\n        res = open_receptacle(loc_to_check)\n    if \"desklamp\" in res:\n        use(res)\n        break"
}
task_ids = ['put','heat','cool','clean','examine']
results = []
with open('./data/execode_plan_prompts/alfworld_plan_extraction_prompt.json', 'w') as f:
    for id in task_ids:
        icl_example_demo = icl_example.replace("<task>",example_task[id]).replace("<traj>",example_traj[id]).replace("<plan>",example_plan[id])
        print(icl_example_demo)
        result = {"task_id":id,"example_task":example_task[id],"example_traj":example_traj[id], "example_plan":example_plan[id],"icl_example":icl_example_demo}
        results.append(result)
    json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
icl_examples = {}
with open('./data/execode_plan_prompts/alfworld_plan_extraction_prompt.json', encoding="utf-8") as f:
    data = json.load(f)
    for line in data:
        icl_examples[line['task_id']]=line['icl_example']

data_path = "./data/ReAct_style_data/alfworld_sft.json" 
output_path = "./data/alfworld_sft_with_execode_plan.json"
with open(output_path,'w',encoding='utf-8') as outf:
    results = []
    with open(data_path, encoding="utf-8") as f:
        data=json.load(f)
        for i, line in enumerate(data):
            id = line['id']
            game_file = line['game_file']
            convs = line['conversations']
            sys_prompt = "[Task Description]:\n" + convs[0]['value']
            task = "\n\n[Task]:\n" + convs[2]['value']
            task_id = getTaskType(task)
            icl_example = icl_examples[task_id]
            trajs = "\n\n[Solution Trajectory]:\n"
            for conv in convs[3:]:
                trajs += conv['value'] + "\n"
            icl_example = icl_example.replace("<sys_prompt>",sys_prompt)
            query = sys_prompt + task + trajs + "\n[Plan]:"
            input_prompt = extract_plan_prompt + icl_example + query
            # print(input_prompt)
            response = llm._call(input_prompt)
            plan = "Thought: First, I devise a plan for solving the task:\n"+response
            line['conversations'][3]['value'] = plan + "\n" + line['conversations'][3]['value'][9:]
            results.append(line)
    json.dump(results, outf, ensure_ascii=False, indent=4)